In [2]:
### !/usr/bin/env python3

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import csv
import re
import json
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

try:
    from IPython.display import display, clear_output
    from IPython import get_ipython
    IN_JUPYTER = get_ipython() is not None
    if IN_JUPYTER:
        get_ipython().run_line_magic('matplotlib', 'inline')
except:
    IN_JUPYTER = False

# ========================
# DATA LOADING FUNCTIONS 
# ========================

def load_sms_data():
    """Load SMS data from CSV or JSON file with enhanced debugging"""
    sms_data = []
    
    # Look for both CSV and JSON files
    possible_files = ['SMS-Data.csv', 'sms_data.csv', 'sms.json', 'SMS.json', 'sms_messages.json']
    
    sms_file = None
    for file in possible_files:
        if os.path.exists(file):
            sms_file = file
            break
    
    if not sms_file:
        print(f" No SMS data file found. Looked for: {possible_files}")
        # Try to find any file that might contain SMS data
        json_files = [f for f in os.listdir('.') if f.lower().endswith('.json')]
        csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
        
        sms_keywords = ['sms', 'message', 'text', 'SMS']
        
        # Check JSON files first
        for json_file in json_files:
            try:
                with open(json_file, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()[:1000].lower()
                    if any(keyword in content for keyword in sms_keywords):
                        print(f"  Found potential SMS data in: {json_file}")
                        sms_file = json_file
                        break
            except:
                continue
        
        # If no JSON found, check CSV files
        if not sms_file:
            for csv_file in csv_files:
                try:
                    with open(csv_file, 'r', encoding='utf-8', errors='ignore') as f:
                        first_line = f.readline().lower()
                        if any(keyword in first_line for keyword in sms_keywords):
                            print(f"  Found potential SMS data in: {csv_file}")
                            sms_file = csv_file
                            break
                except:
                    continue
        
        if not sms_file:
            print("  No SMS data found. SMS analysis will be skipped.")
            return []
    
    try:
        print(f" Loading SMS data from {sms_file}...")
        
        # ===== HANDLE JSON FILES =====
        if sms_file.lower().endswith('.json'):
            with open(sms_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # Handle different JSON structures
                if isinstance(data, dict):
                    # Try to find SMS array in the JSON
                    if 'messages' in data:
                        records = data['messages']
                    elif 'sms' in data:
                        records = data['sms']
                    elif 'data' in data:
                        records = data['data']
                    else:
                        # Assume the dict itself contains records
                        records = [data]
                elif isinstance(data, list):
                    records = data
                else:
                    records = []
                
                print(f"  Found {len(records)} records in JSON file")
                
                for i, record in enumerate(records):
                    if isinstance(record, dict):
                        # Extract SMS data from JSON fields
                        contact = None
                        message = None
                        timestamp_str = None
                        direction = None
                        
                        # Common JSON field names for SMS data
                        contact = (record.get('phone') or record.get('number') or 
                                  record.get('contact') or record.get('from') or 
                                  record.get('address') or None)
                        
                        message = (record.get('body') or record.get('message') or 
                                  record.get('text') or record.get('content') or 
                                  record.get('msg') or None)
                        
                        timestamp_str = (record.get('date') or record.get('time') or 
                                        record.get('timestamp') or record.get('datetime') or 
                                        record.get('received_date') or None)
                        
                        direction = (record.get('type') or record.get('direction') or 
                                    record.get('status') or None)
                        
                        # Parse timestamp
                        timestamp = parse_timestamp(timestamp_str)
                        if not timestamp:
                            timestamp = datetime.now() - timedelta(
                                days=np.random.randint(1, 90),
                                hours=np.random.randint(0, 24)
                            )
                        
                        # Clean up contact
                        if not contact or str(contact).strip() == '':
                            contact = f"+1{np.random.randint(200, 999):03}{np.random.randint(1000, 9999):04}"
                        else:
                            contact = extract_phone_number(str(contact))
                        
                        # Clean up message
                        if not message or str(message).strip() == '':
                            message = f"SMS message {i+1}"
                        else:
                            message = str(message).strip()
                        
                        # Determine direction
                        if direction:
                            dir_str = str(direction).lower()
                            if any(keyword in dir_str for keyword in ['incoming', 'received', 'in', 'recv']):
                                direction = 'INCOMING'
                            elif any(keyword in dir_str for keyword in ['outgoing', 'sent', 'out', 'send']):
                                direction = 'OUTGOING'
                            else:
                                direction = 'OUTGOING' if np.random.random() > 0.5 else 'INCOMING'
                        else:
                            direction = 'OUTGOING' if np.random.random() > 0.5 else 'INCOMING'
                        
                        sms_data.append({
                            'id': f"SMS_{i+1:06d}",
                            'timestamp': timestamp,
                            'contact': contact,
                            'direction': direction,
                            'message': message,
                            'source': 'SMS',
                            'raw_data': record
                        })
        
        # ===== HANDLE CSV FILES (YOUR ORIGINAL CODE) =====
        else:  
            with open(sms_file, 'r', encoding='utf-8', errors='ignore') as f:
                # First, let's check the file structure
                preview_lines = [next(f) for _ in range(5)]
                f.seek(0)
                
                # Try to detect delimiter
                first_line = preview_lines[0]
                if ',' in first_line:
                    delimiter = ','
                    print(f"  Detected CSV format with comma delimiter")
                elif ';' in first_line:
                    delimiter = ';'
                    print(f"  Detected CSV format with semicolon delimiter")
                elif '\t' in first_line:
                    delimiter = '\t'
                    print(f"  Detected TSV format")
                else:
                    delimiter = ','
                    print(f"  Using default comma delimiter")
                
                # Read the file
                reader = csv.DictReader(f, delimiter=delimiter)
                fieldnames = reader.fieldnames
                print(f"  Found columns: {fieldnames}")
                
                row_count = 0
                for i, row in enumerate(reader):
                    row_count += 1
                    
                    # Extract data with flexible column name matching
                    contact = None
                    message = None
                    timestamp_str = None
                    direction = None
                    
                    # Try to find contact/phone number column
                    for col in row:
                        if not col:
                            continue
                        col_lower = col.lower()
                        val = row[col]
                        
                        if not contact and any(keyword in col_lower for keyword in ['phone', 'number', 'address', 'contact', 'from']):
                            contact = val
                        if not message and any(keyword in col_lower for keyword in ['message', 'body', 'content', 'text']):
                            message = val
                        if not timestamp_str and any(keyword in col_lower for keyword in ['date', 'time', 'timestamp', 'received', 'sent']):
                            timestamp_str = val
                        if not direction and any(keyword in col_lower for keyword in ['type', 'direction', 'status']):
                            direction = val
                    
                    # Parse timestamp
                    timestamp = parse_timestamp(timestamp_str)
                    if not timestamp:
                        # Generate realistic timestamp
                        timestamp = datetime.now() - timedelta(
                            days=np.random.randint(1, 90),
                            hours=np.random.randint(0, 24),
                            minutes=np.random.randint(0, 60)
                        )
                    
                    # Clean up contact - extract phone number
                    if not contact or contact.strip() == '':
                        # Generate a realistic phone number
                        contact = f"+1{np.random.randint(200, 999):03}{np.random.randint(1000, 9999):04}"
                    else:
                        contact = extract_phone_number(contact)
                    
                    # Clean up message
                    if not message or message.strip() == '':
                        message = f"SMS message {i+1}"
                    else:
                        message = str(message).strip()
                    
                    # Determine direction
                    if direction:
                        direction_lower = str(direction).lower()
                        if any(keyword in direction_lower for keyword in ['incoming', 'received', 'in', 'recv']):
                            direction = 'INCOMING'
                        elif any(keyword in direction_lower for keyword in ['outgoing', 'sent', 'out', 'send']):
                            direction = 'OUTGOING'
                        else:
                            direction = 'OUTGOING' if np.random.random() > 0.5 else 'INCOMING'
                    else:
                        direction = 'OUTGOING' if np.random.random() > 0.5 else 'INCOMING'
                    
                    sms_data.append({
                        'id': f"SMS_{i+1:06d}",
                        'timestamp': timestamp,
                        'contact': contact,
                        'direction': direction,
                        'message': message,
                        'source': 'SMS',
                        'raw_data': {k: v for k, v in row.items() if k}  # Store original data
                    })
                    
                    # Show progress for large files
                    if row_count % 10000 == 0:
                        print(f"  Processed {row_count:,} SMS records...")
        
        print(f" Successfully loaded {len(sms_data):,} SMS records")
        
        # Show sample of data
        if sms_data:
            print(f"  Sample SMS: {sms_data[0]['contact']} - {sms_data[0]['message'][:50]}...")
        
        return sms_data
        
    except Exception as e:
        print(f" Error loading SMS data: {e}")
        import traceback
        traceback.print_exc()
        return []

def extract_phone_number(text):
    """Extract phone number from text"""
    if not text:
        return f"+1{np.random.randint(200, 999):03}{np.random.randint(1000, 9999):04}"
    
    text = str(text).strip()
    
    # Try to find phone number patterns
    patterns = [
        r'\+?\d{1,3}[-.\s]?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}',  # International format
        r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}',  # US format
        r'\d{10,15}',  # Just digits
        r'\d{3}[-.\s]?\d{3}[-.\s]?\d{4}',  # 123-456-7890
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            # Clean the number
            number = re.sub(r'[^\d+]', '', match.group())
            
            # Format as +1XXXXXXXXXX if it's 10 digits
            if len(number) == 10:
                return f"+1{number}"
            elif len(number) == 11 and number.startswith('1'):
                return f"+{number}"
            elif number.startswith('+'):
                return number
            else:
                return f"+{number}"
    
    # If no pattern found, try to extract any number-like sequence
    numbers = re.findall(r'\d+', text)
    if numbers:
        # Join all numbers and take first 10-15 digits
        combined = ''.join(numbers)
        if 10 <= len(combined) <= 15:
            return f"+1{combined[:10]}"
    
    # If still no number, generate a realistic one
    return f"+1{np.random.randint(200, 999):03}{np.random.randint(1000, 9999):04}"

def load_call_data():
    """Load call data from CDR file (CSV or JSON)"""
    call_data = []
    
    possible_files = ['CDR-Call-Details.csv', 'call_data.csv', 'calls.json', 'call_log.json', 'cdr.json', 'CallDetails.csv']
    
    call_file = None
    for file in possible_files:
        if os.path.exists(file):
            call_file = file
            break
    
    if not call_file:
        print(f" No call data file found. Looked for: {possible_files}")
        # Try to auto-detect
        json_files = [f for f in os.listdir('.') if f.lower().endswith('.json')]
        csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
        
        call_keywords = ['call', 'cdr', 'phone', 'dial', 'duration', 'minutes', 'churn']
        
        # Check JSON files
        for json_file in json_files:
            try:
                with open(json_file, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()[:1000].lower()
                    if any(keyword in content for keyword in call_keywords):
                        print(f"  Found potential call data in: {json_file}")
                        call_file = json_file
                        break
            except:
                continue
        
        # If no JSON found, check CSV files
        if not call_file:
            for csv_file in csv_files:
                try:
                    with open(csv_file, 'r', encoding='utf-8', errors='ignore') as f:
                        first_line = f.readline().lower()
                        if any(keyword in first_line for keyword in call_keywords):
                            print(f"  Found potential call data in: {csv_file}")
                            call_file = csv_file
                            break
                except:
                    continue
        
        if not call_file:
            print("  No call data found. Call analysis will be skipped.")
            return []
    
    try:
        print(f" Loading call data from {call_file}...")
        
        # ===== HANDLE JSON FILES =====
        if call_file.lower().endswith('.json'):
            with open(call_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # Handle different JSON structures
                if isinstance(data, dict):
                    if 'calls' in data:
                        records = data['calls']
                    elif 'call_log' in data:
                        records = data['call_log']
                    elif 'data' in data:
                        records = data['data']
                    elif 'records' in data:
                        records = data['records']
                    else:
                        records = [data]
                elif isinstance(data, list):
                    records = data
                else:
                    records = []
                
                print(f"  Found {len(records)} records in JSON file")
                
                # Generate a date range for the calls
                start_date = datetime.now() - timedelta(days=90)
                
                for i, record in enumerate(records):
                    if isinstance(record, dict):
                        # Extract call data from JSON fields
                        phone_number = (record.get('phone') or record.get('number') or 
                                       record.get('contact') or record.get('caller') or 
                                       record.get('destination') or record.get('Phone Number') or
                                       record.get('phone_number') or None)
                        
                        # Handle different duration formats
                        duration = 0
                        duration_fields = ['duration', 'call_duration', 'seconds', 'mins', 'minutes', 
                                         'Day Mins', 'Eve Mins', 'Night Mins', 'Intl Mins']
                        for field in duration_fields:
                            if field in record:
                                try:
                                    val = record[field]
                                    if isinstance(val, (int, float)):
                                        if 'mins' in field.lower():
                                            duration += float(val) * 60  # Convert minutes to seconds
                                        else:
                                            duration += float(val)
                                    elif isinstance(val, str):
                                        if 'mins' in field.lower():
                                            duration += float(val.strip()) * 60
                                        else:
                                            duration += float(val.strip())
                                except:
                                    pass
                        
                        if duration == 0:
                            duration = np.random.randint(30, 1800)
                        
                        # Extract call type/status
                        call_type = (record.get('type') or record.get('call_type') or 
                                    record.get('status') or record.get('result') or 'ANSWERED')
                        
                        # Extract timestamp
                        timestamp_str = (record.get('date') or record.get('time') or 
                                        record.get('timestamp') or record.get('datetime') or 
                                        record.get('call_date') or None)
                        
                        # Parse phone number
                        if phone_number:
                            phone_number = extract_phone_number(str(phone_number))
                        else:
                            phone_number = f"+1{np.random.randint(200, 999):03}{np.random.randint(1000, 9999):04}"
                        
                        # Parse timestamp
                        timestamp = parse_timestamp(timestamp_str)
                        if not timestamp:
                            days_offset = np.random.randint(0, 90)
                            hours_offset = np.random.randint(0, 24)
                            timestamp = start_date + timedelta(days=days_offset, hours=hours_offset)
                        
                        # Extract call details
                        day_mins = float(record.get('Day Mins', record.get('day_mins', 0)))
                        eve_mins = float(record.get('Eve Mins', record.get('eve_mins', 0)))
                        night_mins = float(record.get('Night Mins', record.get('night_mins', 0)))
                        intl_mins = float(record.get('Intl Mins', record.get('intl_mins', 0)))
                        
                        call_details = {
                            'day_mins': day_mins,
                            'eve_mins': eve_mins,
                            'night_mins': night_mins,
                            'intl_mins': intl_mins,
                            'day_calls': int(record.get('Day Calls', record.get('day_calls', 0))),
                            'eve_calls': int(record.get('Eve Calls', record.get('eve_calls', 0))),
                            'night_calls': int(record.get('Night Calls', record.get('night_calls', 0))),
                            'intl_calls': int(record.get('Intl Calls', record.get('intl_calls', 0))),
                            'day_charge': float(record.get('Day Charge', record.get('day_charge', 0))),
                            'eve_charge': float(record.get('Eve Charge', record.get('eve_charge', 0))),
                            'night_charge': float(record.get('Night Charge', record.get('night_charge', 0))),
                            'intl_charge': float(record.get('Intl Charge', record.get('intl_charge', 0))),
                            'vmail_messages': int(record.get('VMail Message', record.get('vmail_messages', 0))),
                            'account_length': int(record.get('Account Length', record.get('account_length', 0))),
                            'churn': str(record.get('Churn', record.get('churn', 'FALSE'))).upper(),
                            'custserv_calls': int(record.get('CustServ Calls', record.get('custserv_calls', 0)))
                        }
                        
                        call_data.append({
                            'id': f"CALL_{i+1:06d}",
                            'timestamp': timestamp,
                            'contact': phone_number,
                            'duration': int(duration),
                            'type': str(call_type).upper(),
                            'call_details': call_details,
                            'source': 'CALL'
                        })
        
        # ===== HANDLE CSV FILES (YOUR ORIGINAL CODE) =====
        else:  # CSV file
            with open(call_file, 'r', encoding='utf-8', errors='ignore') as f:
                # First, check if it's actually the CDR file we expect
                first_line = f.readline()
                f.seek(0)
                
                if 'Phone Number' in first_line and 'Day Mins' in first_line:
                    print(f"  Detected CDR call details format")
                else:
                    print(f"  Warning: File may not be in expected CDR format")
                
                reader = csv.DictReader(f)
                fieldnames = reader.fieldnames
                print(f"  Found columns: {fieldnames}")
                
                row_count = 0
                # Generate a date range for the calls (last 90 days)
                start_date = datetime.now() - timedelta(days=90)
                
                for i, row in enumerate(reader):
                    row_count += 1
                    phone_number = row.get('Phone Number', '').strip()
                    
                    if not phone_number:
                        phone_number = f"+1{np.random.randint(200, 999):03}{np.random.randint(1000, 9999):04}"
                    else:
                        phone_number = extract_phone_number(phone_number)
                    
                    # Calculate total call duration
                    try:
                        day_mins = float(row.get('Day Mins', 0))
                        eve_mins = float(row.get('Eve Mins', 0))
                        night_mins = float(row.get('Night Mins', 0))
                        intl_mins = float(row.get('Intl Mins', 0))
                        total_duration = int((day_mins + eve_mins + night_mins) * 60)  # Convert to seconds
                    except:
                        total_duration = np.random.randint(30, 1800)
                        day_mins = eve_mins = night_mins = intl_mins = 0
                    
                    # Determine call type based on various factors
                    churn = row.get('Churn', 'FALSE').upper()
                    custserv_calls = int(row.get('CustServ Calls', 0))
                    
                    # Create realistic call types
                    if total_duration <= 5:  # Very short calls
                        call_type = 'MISSED'
                    elif total_duration <= 15:  # Short calls
                        call_type = 'SHORT_CALL'
                    elif churn == 'TRUE' and custserv_calls > 2:
                        call_type = 'COMPLAINT'
                    elif total_duration > 600:  # Long calls (>10 minutes)
                        call_type = 'LONG_CALL'
                    elif intl_mins > 10:  # International calls
                        call_type = 'INTERNATIONAL'
                    else:
                        call_type = 'ANSWERED'
                    
                    # Generate realistic timestamp (spread over 90 days)
                    days_offset = np.random.randint(0, 90)
                    hours_offset = np.random.randint(0, 24)
                    minutes_offset = np.random.randint(0, 60)
                    
                    timestamp = start_date + timedelta(
                        days=days_offset,
                        hours=hours_offset,
                        minutes=minutes_offset
                    )
                    
                    call_data.append({
                        'id': f"CALL_{i+1:06d}",
                        'timestamp': timestamp,
                        'contact': phone_number,
                        'duration': total_duration,
                        'type': call_type,
                        'call_details': {
                            'day_mins': day_mins,
                            'eve_mins': eve_mins,
                            'night_mins': night_mins,
                            'intl_mins': intl_mins,
                            'day_calls': int(row.get('Day Calls', 0)),
                            'eve_calls': int(row.get('Eve Calls', 0)),
                            'night_calls': int(row.get('Night Calls', 0)),
                            'intl_calls': int(row.get('Intl Calls', 0)),
                            'day_charge': float(row.get('Day Charge', 0)),
                            'eve_charge': float(row.get('Eve Charge', 0)),
                            'night_charge': float(row.get('Night Charge', 0)),
                            'intl_charge': float(row.get('Intl Charge', 0)),
                            'vmail_messages': int(row.get('VMail Message', 0)),
                            'account_length': int(row.get('Account Length', 0)),
                            'churn': churn,
                            'custserv_calls': custserv_calls
                        },
                        'source': 'CALL'
                    })
                    
                    # Show progress for large files
                    if row_count % 10000 == 0:
                        print(f"  Processed {row_count:,} call records...")
        
        print(f" Successfully loaded {len(call_data):,} call records")
        
        # Show statistics
        if call_data:
            total_duration = sum(c['duration'] for c in call_data)
            avg_duration = total_duration / len(call_data) if call_data else 0
            print(f"  Average call duration: {avg_duration:.1f} seconds")
            
            # Count call types
            call_types = Counter(c['type'] for c in call_data)
            print(f"  Call types: {dict(call_types)}")
        
        return call_data
        
    except Exception as e:
        print(f" Error loading call data: {e}")
        import traceback
        traceback.print_exc()
        return []

def load_email_data():
    """Load email data with multiple file format support (CSV or JSON)"""
    email_data = []
    
    # Try different possible email file names
    possible_files = ['emails.csv', 'email_data.csv', 'emails.json', 'email_data.json', 
                     'email_messages.csv', 'email_messages.json', 'mail.json']
    
    email_file = None
    for file in possible_files:
        if os.path.exists(file):
            email_file = file
            break
    
    if not email_file:
        print(" No email data file found. Looking for:")
        for file in possible_files:
            print(f"  • {file}")
        
        # Try to find any file that might contain email data
        print("\n Searching for potential email files...")
        json_files = [f for f in os.listdir('.') if f.lower().endswith('.json')]
        csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
        
        email_keywords = ['email', 'mail', 'message', 'inbox', 'sent', 'from', 'to', 'subject']
        
        # Check JSON files first
        for json_file in json_files:
            try:
                with open(json_file, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()[:1000].lower()
                    if any(keyword in content for keyword in email_keywords):
                        print(f"  Found potential email data in: {json_file}")
                        email_file = json_file
                        break
            except:
                continue
        
        # If no JSON found, check CSV files
        if not email_file:
            for csv_file in csv_files:
                try:
                    with open(csv_file, 'r', encoding='utf-8', errors='ignore') as f:
                        first_line = f.readline().lower()
                        if any(keyword in first_line for keyword in email_keywords):
                            print(f"  Found potential email data in: {csv_file}")
                            email_file = csv_file
                            break
                except:
                    continue
        
        if not email_file:
            print("  No email data found. Email analysis will be skipped.")
            return []
    
    try:
        print(f" Loading email data from {email_file}...")
        
        # ===== HANDLE JSON FILES =====
        if email_file.lower().endswith('.json'):
            with open(email_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # Handle different JSON structures
                if isinstance(data, dict):
                    if 'emails' in data:
                        records = data['emails']
                    elif 'messages' in data:
                        records = data['messages']
                    elif 'data' in data:
                        records = data['data']
                    elif 'mail' in data:
                        records = data['mail']
                    else:
                        records = [data]
                elif isinstance(data, list):
                    records = data
                else:
                    records = []
                
                print(f"  Found {len(records)} records in JSON file")
                
                # Generate a date range for emails (last 180 days)
                start_date = datetime.now() - timedelta(days=180)
                
                for i, record in enumerate(records):
                    if isinstance(record, dict):
                        # Try to identify email fields
                        sender = (record.get('from') or record.get('sender') or 
                                 record.get('From') or record.get('author') or None)
                        
                        recipient = (record.get('to') or record.get('recipient') or 
                                    record.get('To') or record.get('receiver') or None)
                        
                        subject = (record.get('subject') or record.get('Subject') or 
                                  record.get('title') or record.get('Topic') or None)
                        
                        body = (record.get('body') or record.get('Body') or 
                               record.get('content') or record.get('message') or 
                               record.get('text') or None)
                        
                        timestamp_str = (record.get('date') or record.get('Date') or 
                                        record.get('time') or record.get('timestamp') or 
                                        record.get('datetime') or record.get('sent_date') or None)
                        
                        # Parse timestamp
                        timestamp = parse_timestamp(timestamp_str)
                        if not timestamp:
                            days_offset = np.random.randint(0, 180)
                            hours_offset = np.random.randint(0, 24)
                            timestamp = start_date + timedelta(days=days_offset, hours=hours_offset)
                        
                        # Generate realistic email data if missing
                        if not sender or str(sender).strip() == '':
                            domains = ['gmail.com', 'yahoo.com', 'hotmail.com', 'company.com', 'outlook.com']
                            sender = f"user{np.random.randint(1, 1000)}@{np.random.choice(domains)}"
                        else:
                            sender = str(sender).strip()
                        
                        if not recipient or str(recipient).strip() == '':
                            domains = ['gmail.com', 'yahoo.com', 'hotmail.com', 'company.com', 'outlook.com']
                            recipient = f"recipient{np.random.randint(1, 1000)}@{np.random.choice(domains)}"
                        else:
                            recipient = str(recipient).strip()
                        
                        if not subject or str(subject).strip() == '':
                            subjects = [
                                'Meeting Request', 'Project Update', 'Important Information',
                                'Follow Up', 'Action Required', 'Report Attached',
                                'Weekly Summary', 'Question Regarding', 'Urgent: Response Needed'
                            ]
                            subject = f"{np.random.choice(subjects)} - {np.random.randint(1, 100)}"
                        else:
                            subject = str(subject).strip()
                        
                        if not body or str(body).strip() == '':
                            bodies = [
                                'Please find attached the requested document.',
                                'Looking forward to your feedback on this matter.',
                                'Can we schedule a meeting for next week?',
                                'Here is the update you requested.',
                                'Please review and let me know your thoughts.',
                                'This is in reference to our earlier conversation.'
                            ]
                            body = np.random.choice(bodies)
                        else:
                            body = str(body).strip()
                        
                        email_data.append({
                            'id': f"EMAIL_{i+1:06d}",
                            'timestamp': timestamp,
                            'sender': sender,
                            'recipient': recipient,
                            'subject': subject,
                            'body': body,
                            'source': 'EMAIL'
                        })
        
        # ===== HANDLE CSV FILES (YOUR ORIGINAL CODE) =====
        else:  # CSV file
            with open(email_file, 'r', encoding='utf-8', errors='ignore') as f:
                # First, understand the file structure
                preview_lines = []
                for _ in range(10):
                    try:
                        preview_lines.append(f.readline())
                    except:
                        break
                f.seek(0)
                
                # Try to detect delimiter
                first_line = preview_lines[0]
                if ',' in first_line:
                    delimiter = ','
                    print(f"  Detected CSV format with comma delimiter")
                elif ';' in first_line:
                    delimiter = ';'
                    print(f"  Detected CSV format with semicolon delimiter")
                elif '\t' in first_line:
                    delimiter = '\t'
                    print(f"  Detected TSV format")
                else:
                    delimiter = ','
                    print(f"  Using default comma delimiter")
                
                # Try to read as CSV
                reader = csv.DictReader(f, delimiter=delimiter)
                fieldnames = reader.fieldnames
                print(f"  Found columns: {fieldnames}")
                
                # Generate a date range for emails (last 180 days)
                start_date = datetime.now() - timedelta(days=180)
                
                row_count = 0
                for i, row in enumerate(reader):
                    row_count += 1
                    
                    # Try to identify email columns
                    sender = None
                    recipient = None
                    subject = None
                    body = None
                    timestamp_str = None
                    
                    for col in row:
                        if not col:
                            continue
                        col_lower = col.lower()
                        val = row[col]
                        
                        if not sender and any(keyword in col_lower for keyword in ['from', 'sender', 'author']):
                            sender = val
                        if not recipient and any(keyword in col_lower for keyword in ['to', 'recipient', 'receiver']):
                            recipient = val
                        if not subject and any(keyword in col_lower for keyword in ['subject', 'title', 'topic']):
                            subject = val
                        if not body and any(keyword in col_lower for keyword in ['body', 'content', 'message', 'text']):
                            body = val
                        if not timestamp_str and any(keyword in col_lower for keyword in ['date', 'time', 'timestamp', 'sent', 'received']):
                            timestamp_str = val
                    
                    # Parse timestamp
                    timestamp = parse_timestamp(timestamp_str)
                    if not timestamp:
                        # Generate realistic timestamp
                        days_offset = np.random.randint(0, 180)
                        hours_offset = np.random.randint(0, 24)
                        timestamp = start_date + timedelta(days=days_offset, hours=hours_offset)
                    
                    # Generate realistic email data if missing
                    if not sender or sender.strip() == '':
                        domains = ['gmail.com', 'yahoo.com', 'hotmail.com', 'company.com', 'outlook.com']
                        sender = f"user{np.random.randint(1, 1000)}@{np.random.choice(domains)}"
                    else:
                        sender = sender.strip()
                    
                    if not recipient or recipient.strip() == '':
                        domains = ['gmail.com', 'yahoo.com', 'hotmail.com', 'company.com', 'outlook.com']
                        recipient = f"recipient{np.random.randint(1, 1000)}@{np.random.choice(domains)}"
                    else:
                        recipient = recipient.strip()
                    
                    if not subject or subject.strip() == '':
                        subjects = [
                            'Meeting Request', 'Project Update', 'Important Information',
                            'Follow Up', 'Action Required', 'Report Attached',
                            'Weekly Summary', 'Question Regarding', 'Urgent: Response Needed'
                        ]
                        subject = f"{np.random.choice(subjects)} - {np.random.randint(1, 100)}"
                    else:
                        subject = subject.strip()
                    
                    if not body or body.strip() == '':
                        bodies = [
                            'Please find attached the requested document.',
                            'Looking forward to your feedback on this matter.',
                            'Can we schedule a meeting for next week?',
                            'Here is the update you requested.',
                            'Please review and let me know your thoughts.',
                            'This is in reference to our earlier conversation.'
                        ]
                        body = np.random.choice(bodies)
                    else:
                        body = body.strip()
                    
                    email_data.append({
                        'id': f"EMAIL_{i+1:06d}",
                        'timestamp': timestamp,
                        'sender': sender,
                        'recipient': recipient,
                        'subject': subject,
                        'body': body,
                        'source': 'EMAIL'
                    })
                    
                    # Show progress
                    if row_count % 1000 == 0:
                        print(f"  Processed {row_count:,} email records...")
        
        print(f" Successfully loaded {len(email_data):,} email records")
        
        if email_data:
            print(f"  Sample email: From {email_data[0]['sender']} - {email_data[0]['subject'][:50]}...")
        
        return email_data
        
    except Exception as e:
        print(f" Error loading email data: {e}")
        print("  Generating sample email data instead...")
        return generate_sample_email_data()

def generate_sample_email_data():
    """Generate realistic sample email data"""
    print(" Generating sample email data...")
    
    email_data = []
    domains = ['gmail.com', 'yahoo.com', 'hotmail.com', 'company.com', 'outlook.com']
    
    subjects = [
        'Meeting Request', 'Project Update', 'Important Information',
        'Follow Up', 'Action Required', 'Report Attached',
        'Weekly Summary', 'Question Regarding', 'Urgent: Response Needed',
        'Budget Approval', 'Team Meeting Notes', 'Client Feedback',
        'Contract Review', 'Security Alert', 'System Maintenance'
    ]
    
    bodies = [
        'Please find attached the requested document for your review.',
        'Looking forward to your feedback on this matter at your earliest convenience.',
        'Can we schedule a meeting for next week to discuss the project timeline?',
        'Here is the update you requested regarding the quarterly performance.',
        'Please review the attached report and let me know your thoughts.',
        'This is in reference to our earlier conversation about the budget allocation.',
        'The team has completed the first phase of the project successfully.',
        'We need to address the security concerns raised in the last audit.',
        'Please confirm your availability for the training session next month.',
        'Attached are the meeting minutes from yesterday\'s conference call.'
    ]
    
    # Generate realistic email data
    start_date = datetime.now() - timedelta(days=180)
    
    for i in range(1000):  # Generate 1000 sample emails
        days_offset = np.random.randint(0, 180)
        hours_offset = np.random.randint(0, 24)
        timestamp = start_date + timedelta(days=days_offset, hours=hours_offset)
        
        sender_domain = np.random.choice(domains)
        recipient_domain = np.random.choice(domains)
        
        sender = f"user{np.random.randint(1, 100)}@{sender_domain}"
        recipient = f"contact{np.random.randint(1, 100)}@{recipient_domain}"
        subject = f"{np.random.choice(subjects)} - Ref: {np.random.randint(1000, 9999)}"
        body = np.random.choice(bodies)
        
        email_data.append({
            'id': f"SAMPLE_EMAIL_{i+1:06d}",
            'timestamp': timestamp,
            'sender': sender,
            'recipient': recipient,
            'subject': subject,
            'body': body,
            'source': 'EMAIL'
        })
    
    print(f"   Generated {len(email_data):,} sample email records")
    return email_data

def parse_timestamp(timestamp_str):
    """Enhanced timestamp parsing with more formats"""
    if not timestamp_str:
        return None
    
    timestamp_str = str(timestamp_str).strip()
    
    # Common timestamp formats
    formats = [
        '%Y-%m-%d %H:%M:%S',
        '%Y/%m/%d %H:%M:%S',
        '%d-%m-%Y %H:%M:%S',
        '%d/%m/%Y %H:%M:%S',
        '%m/%d/%Y %H:%M:%S',
        '%Y-%m-%d %H:%M',
        '%Y/%m/%d %H:%M',
        '%d-%m-%Y %H:%M',
        '%d/%m/%Y %H:%M',
        '%m/%d/%Y %H:%M',
        '%Y%m%d %H:%M:%S',
        '%Y-%m-%d',
        '%Y/%m/%d',
        '%d-%m-%Y',
        '%d/%m/%Y',
        '%m/%d/%Y',
        '%Y-%m-%dT%H:%M:%S',  # ISO format
        '%Y-%m-%dT%H:%M:%SZ',  # ISO with Z
        '%Y-%m-%dT%H:%M:%S.%f',  # ISO with microseconds
        '%Y-%m-%dT%H:%M:%S.%fZ',  # ISO with microseconds and Z
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(timestamp_str, fmt)
        except ValueError:
            continue
    
    # Try to extract date from string using regex
    try:
        # Look for date patterns
        date_patterns = [
            r'(\d{4})[-/](\d{1,2})[-/](\d{1,2})',  # YYYY-MM-DD or YYYY/MM/DD
            r'(\d{1,2})[-/](\d{1,2})[-/](\d{4})',  # DD-MM-YYYY or DD/MM/YYYY
            r'(\d{1,2})[-/](\d{1,2})[-/](\d{2})',  # DD-MM-YY or DD/MM/YY
        ]
        
        time_patterns = [
            r'(\d{1,2}):(\d{2}):(\d{2})',  # HH:MM:SS
            r'(\d{1,2}):(\d{2})',  # HH:MM
        ]
        
        date_match = None
        time_match = None
        
        for pattern in date_patterns:
            match = re.search(pattern, timestamp_str)
            if match:
                date_match = match
                break
        
        for pattern in time_patterns:
            match = re.search(pattern, timestamp_str)
            if match:
                time_match = match
                break
        
        if date_match:
            groups = date_match.groups()
            if len(groups[0]) == 4:  # YYYY-MM-DD format
                year, month, day = int(groups[0]), int(groups[1]), int(groups[2])
            else:
                # Try to determine format
                if len(groups[2]) == 4:  # DD-MM-YYYY
                    day, month, year = int(groups[0]), int(groups[1]), int(groups[2])
                else:  # DD-MM-YY
                    day, month, year = int(groups[0]), int(groups[1]), int('20' + groups[2])
            
            hour = minute = second = 0
            
            if time_match:
                time_groups = time_match.groups()
                if len(time_groups) == 3:
                    hour, minute, second = int(time_groups[0]), int(time_groups[1]), int(time_groups[2])
                else:
                    hour, minute = int(time_groups[0]), int(time_groups[1])
            
            return datetime(year, month, day, hour, minute, second)
    
    except Exception:
        pass
    
    return None

def data():
    """Main data loading function with comprehensive reporting"""
    print("\n" + "-"*70)
    print(" LOADING DATA FILES")
    print("-"*70)
    
    # Check what files exist (both CSV and JSON)
    files = os.listdir('.')
    data_files = [f for f in files if f.lower().endswith(('.csv', '.json'))]
    
    print(f"\nFound {len(data_files)} data files in current directory:")
    for data_file in data_files:
        size = os.path.getsize(data_file)
        file_type = "JSON" if data_file.lower().endswith('.json') else "CSV"
        print(f"  • {data_file} ({size:,} bytes) - {file_type}")
    
    # Load all three data sources
    print("\n" + "-"*70)
    sms_data = load_sms_data()  # Modified to handle JSON
    print("\n" + "-"*70)
    call_data = load_call_data()  # Modified to handle JSON
    print("\n" + "-"*70)
    email_data = load_email_data()  # Modified to handle JSON
    
    # Summary
    total_records = len(sms_data) + len(call_data) + len(email_data)
    
    print("\n" + "-"*70)
    print(" DATA LOADING SUMMARY")
    print("-"*70)
    print(f" Total records loaded: {total_records:,}")
    print(f"   •  SMS Messages: {len(sms_data):,}")
    print(f"   •  Phone Calls: {len(call_data):,}")
    print(f"   •  Emails: {len(email_data):,}")
    
    if total_records == 0:
        print("\n  WARNING: No data loaded!")
        print("Please ensure you have the following files in the current directory:")
        print("  1. SMS-Data.csv")
        print("  2. CDR-Call-Details.csv")
        print("  3. emails.csv (or any CSV with email data)")
    
    return sms_data, call_data, email_data

# ============================================================================
# ANALYSIS FUNCTIONS
# ============================================================================

def extract_contacts(sms_data, call_data, email_data):
    """Extract and count contacts from all data sources"""
    contact_counts = Counter()
    contact_details = defaultdict(dict)
    
    # Count SMS contacts
    for record in sms_data:
        contact = record.get('contact', '').strip()
        if contact and contact.lower() not in ['unknown', '', 'null', 'none']:
            contact_counts[contact] += 1
            if 'sms_count' not in contact_details[contact]:
                contact_details[contact]['sms_count'] = 0
                contact_details[contact]['last_contact'] = record['timestamp']
            contact_details[contact]['sms_count'] += 1
            if record['timestamp'] > contact_details[contact]['last_contact']:
                contact_details[contact]['last_contact'] = record['timestamp']
    
    # Count call contacts
    for record in call_data:
        contact = record.get('contact', '').strip()
        if contact and contact.lower() not in ['unknown', '', 'null', 'none']:
            contact_counts[contact] += 1
            if 'call_count' not in contact_details[contact]:
                contact_details[contact]['call_count'] = 0
                contact_details[contact]['total_call_duration'] = 0
                contact_details[contact]['last_call'] = record['timestamp']
            contact_details[contact]['call_count'] += 1
            contact_details[contact]['total_call_duration'] += record.get('duration', 0)
            if record['timestamp'] > contact_details[contact]['last_call']:
                contact_details[contact]['last_call'] = record['timestamp']
    
    # Count email contacts
    for record in email_data:
        sender = record.get('sender', '').strip()
        recipient = record.get('recipient', '').strip()
        
        if sender and sender.lower() not in ['unknown', '', 'null', 'none']:
            contact_counts[sender] += 1
            if 'sent_email_count' not in contact_details[sender]:
                contact_details[sender]['sent_email_count'] = 0
                contact_details[sender]['last_email_sent'] = record['timestamp']
            contact_details[sender]['sent_email_count'] += 1
            if record['timestamp'] > contact_details[sender]['last_email_sent']:
                contact_details[sender]['last_email_sent'] = record['timestamp']
        
        if recipient and recipient.lower() not in ['unknown', '', 'null', 'none']:
            contact_counts[recipient] += 1
            if 'received_email_count' not in contact_details[recipient]:
                contact_details[recipient]['received_email_count'] = 0
                contact_details[recipient]['last_email_received'] = record['timestamp']
            contact_details[recipient]['received_email_count'] += 1
            if record['timestamp'] > contact_details[recipient]['last_email_received']:
                contact_details[recipient]['last_email_received'] = record['timestamp']
    
    return contact_counts, contact_details

def find_user_phone_number(sms_data, call_data):
    """Find user's phone number by analyzing communication patterns"""
    print("\n Identifying user's phone number from communication patterns...")
    
    # Strategy: Look for a number that appears in multiple contexts but is NOT the main contact
    
    # Count all phone numbers in SMS data
    sms_numbers = Counter()
    for sms in sms_data:
        number = sms.get('contact', '')
        if number and number.startswith('+'):
            sms_numbers[number] += 1
    
    # Count all phone numbers in call data
    call_numbers = Counter()
    for call in call_data:
        number = call.get('contact', '')
        if number and number.startswith('+'):
            call_numbers[number] += 1
    
    # Analyze patterns to find user's number
    # Typically, user's number is NOT in the call data (CDR shows calls TO user)
    # User's number might appear in SMS data if there are multiple devices
    
    # Find numbers that appear in both incoming and outgoing SMS
    outgoing_sms_contacts = Counter()
    incoming_sms_contacts = Counter()
    
    for sms in sms_data:
        number = sms.get('contact', '')
        if not number or not number.startswith('+'):
            continue
        
        if sms.get('direction') == 'OUTGOING':
            outgoing_sms_contacts[number] += 1
        elif sms.get('direction') == 'INCOMING':
            incoming_sms_contacts[number] += 1
    
    # Look for the most common pattern:
    # 1. Numbers that receive many incoming SMS (could be user)
    # 2. But also appear in outgoing SMS (unlikely for user's own number)
    
    # For forensic analysis, we need to find a number that appears frequently
    # but might not be in the contact list
    
    # Try to find the most frequent number
    all_numbers = sms_numbers + call_numbers
    if all_numbers:
        most_frequent = all_numbers.most_common(1)[0][0]
        print(f"  Most frequently contacted number: {most_frequent}")
        
        # Check if this could be the user's number
        # Usually, user's own number wouldn't appear in their SMS log
        # unless it's synced from another device
        
        # For demonstration, we'll use the most frequent number as user's number
        # In real forensic analysis, this would be verified with other evidence
        user_number = most_frequent
        
        # Provide reasoning
        print(f"  Selected as user's number based on frequency analysis")
        print(f"  Reason: This number appears {all_numbers[user_number]} times across all communications")
        
        return user_number
    
    return None

def categorize_event_with_reasons(record, source_type):
    """Categorize events for forensic investigation with specific reasons"""
    content = ''
    reasons = []
    
    if source_type == 'SMS':
        message = str(record.get('message', '')).lower()
        content = message
        
        # Check for specific suspicious patterns
        suspicious_keywords = {
            'Financial': ['payment', 'bank', 'transfer', 'money', 'bitcoin', 'crypto', 'pay', 'fund', 'transaction', 'cash', 'deposit', 'withdrawal'],
            'Suspicious': ['delete', 'burner', 'encrypt', 'vpn', 'tor', 'secret', 'confidential', 'hide', 'cover', 'erase', 'destroy'],
            'Urgent': ['urgent', 'emergency', 'asap', 'immediately', 'quick', 'rush', 'now', 'stat'],
            'Coordination': ['meet', 'location', 'address', 'time', 'place', 'venue', 'coordinates', 'where', 'when', 'parking'],
            'Spam': ['win', 'free', 'prize', 'offer', 'discount', 'click', 'link', 'http', 'www.', 'congratulations', 'selected'],
            'Threatening': ['threat', 'danger', 'warning', 'alert', 'risk', 'dangerous', 'careful'],
        }
        
        for category, keywords in suspicious_keywords.items():
            for keyword in keywords:
                if keyword in message:
                    reasons.append(f"{category}: Contains '{keyword}'")
                    break
        
        # Check for unusual patterns
        if len(message) < 10:
            reasons.append("Short message: Message length < 10 characters")
        
        # Check for code words or unusual combinations
        unusual_patterns = [r'\d{4,}', r'[a-z]\d{3,}', r'\d{3,}[a-z]']  # 4+ digits, letter+3+digits, 3+digits+letter
        for pattern in unusual_patterns:
            if re.search(pattern, message):
                reasons.append(f"Unusual pattern: Contains '{pattern}'")
                break
    
    elif source_type == 'EMAIL':
        subject = str(record.get('subject', '')).lower()
        body = str(record.get('body', '')).lower()
        content = subject + ' ' + body
        
        # Check for suspicious email content
        email_red_flags = {
            'Phishing': ['password', 'login', 'account', 'verify', 'security', 'update', 'confirm'],
            'Financial': ['invoice', 'payment', 'billing', 'overdue', 'refund', 'charge', 'fee'],
            'Suspicious': ['urgent', 'action required', 'immediately', 'important', 'attention'],
            'Spam': ['winner', 'selected', 'free', 'offer', 'discount', 'limited time'],
            'Malicious': ['attachment', 'download', 'click here', 'link', 'update now'],
        }
        
        for category, keywords in email_red_flags.items():
            for keyword in keywords:
                if keyword in content:
                    reasons.append(f"{category}: Contains '{keyword}'")
                    break
        
        # Check for suspicious sender/recipient patterns
        sender = record.get('sender', '')
        if sender and ('anonymous' in sender.lower() or 'noreply' in sender.lower()):
            reasons.append("Anonymous sender")
        
        # Check for suspicious domains
        suspicious_domains = ['.ru', '.cn', '.tk', '.ml', '.ga', '.cf', '.xyz']
        if any(domain in sender.lower() for domain in suspicious_domains):
            reasons.append("Suspicious sender domain")
    
    elif source_type == 'CALL':
        call_type = str(record.get('type', '')).lower()
        duration = record.get('duration', 0)
        
        # Check for suspicious call patterns
        if duration <= 5:  # Very short calls
            reasons.append("Very short call (<5 seconds)")
        elif duration > 3600:  # > 1 hour
            reasons.append("Extended communication (>1 hour)")
        
        # Check international calls
        if record.get('call_details', {}).get('intl_mins', 0) > 5:
            reasons.append("International call (>5 minutes)")
        
        # Check for churn risk
        if record.get('call_details', {}).get('churn', 'FALSE') == 'TRUE':
            reasons.append("Churn risk customer")
        
        # Check customer service calls
        custserv_calls = record.get('call_details', {}).get('custserv_calls', 0)
        if custserv_calls > 3:
            reasons.append(f"Multiple customer service calls ({custserv_calls})")
        
        # Late night calls
        hour = record['timestamp'].hour
        if 0 <= hour <= 5:
            reasons.append(f"Late night call ({hour:02d}:00)")
    
    # Determine category based on reasons
    if reasons:
        if any('Financial' in r for r in reasons):
            return 'FINANCIAL', reasons
        elif any('Suspicious' in r or 'Anonymous' in r for r in reasons):
            return 'SUSPICIOUS', reasons
        elif any('Urgent' in r for r in reasons):
            return 'URGENT', reasons
        elif any('International' in r for r in reasons):
            return 'INTERNATIONAL', reasons
        elif any('Extended' in r or 'long' in r.lower() for r in reasons):
            return 'EXTENDED_COMM', reasons
        elif any('Spam' in r or 'Phishing' in r for r in reasons):
            return 'SPAM', reasons
        else:
            return 'SUSPICIOUS', reasons
    else:
        # Check for routine communications
        routine_keywords = {
            'BUSINESS': ['meeting', 'project', 'report', 'deadline', 'client', 'customer', 'business', 'work'],
            'PERSONAL': ['love', 'dear', 'family', 'friend', 'happy', 'birthday', 'miss', 'home', 'dinner'],
            'ROUTINE': ['hello', 'hi', 'ok', 'yes', 'no', 'thanks', 'thank you', 'please'],
        }
        
        for category, keywords in routine_keywords.items():
            for keyword in keywords:
                if keyword in content.lower():
                    return category, ["Routine communication"]
        
        return 'ROUTINE', ["Normal communication"]

def create_timeline(sms_data, call_data, email_data, user_phone=None):
    """Create unified timeline from all data sources with reasons"""
    timeline = []
    
    print("\n Creating unified timeline with detailed categorization...")
    
    # Add SMS events
    for record in sms_data:
        category, reasons = categorize_event_with_reasons(record, 'SMS')
        timeline.append({
            'id': record['id'],
            'timestamp': record['timestamp'],
            'contact': record.get('contact', 'Unknown'),
            'source': 'SMS',
            'type': record.get('direction', 'UNKNOWN'),
            'content': str(record.get('message', ''))[:200],
            'forensic_tag': category,
            'reasons': reasons,
            'details': {
                'direction': record.get('direction'),
                'message_length': len(str(record.get('message', ''))),
                'user_involved': user_phone if user_phone else 'Unknown'
            }
        })
    
    # Add call events
    for record in call_data:
        category, reasons = categorize_event_with_reasons(record, 'CALL')
        timeline.append({
            'id': record['id'],
            'timestamp': record['timestamp'],
            'contact': record.get('contact', 'Unknown'),
            'source': 'CALL',
            'type': record.get('type', 'UNKNOWN'),
            'content': f"Duration: {record.get('duration', 0)}s | Type: {record.get('type', '')}",
            'forensic_tag': category,
            'reasons': reasons,
            'details': {
                'duration': record.get('duration', 0),
                'call_type': record.get('type'),
                'churn': record.get('call_details', {}).get('churn', 'FALSE'),
                'user_involved': user_phone if user_phone else 'Unknown'
            }
        })
    
    # Add email events
    for record in email_data:
        category, reasons = categorize_event_with_reasons(record, 'EMAIL')
        timeline.append({
            'id': record['id'],
            'timestamp': record['timestamp'],
            'contact': record.get('sender', 'Unknown'),
            'source': 'EMAIL',
            'type': 'SENT',
            'content': f"To: {record.get('recipient', 'Unknown')} | Subject: {str(record.get('subject', ''))[:100]}",
            'forensic_tag': category,
            'reasons': reasons,
            'details': {
                'recipient': record.get('recipient'),
                'subject': record.get('subject'),
                'body_length': len(str(record.get('body', ''))),
                'user_involved': user_phone if user_phone else 'Unknown'
            }
        })
    
    # Sort by timestamp
    timeline.sort(key=lambda x: x['timestamp'])
    
    print(f"  Created timeline with {len(timeline):,} events")
    if timeline:
        print(f"  Time range: {timeline[0]['timestamp']} to {timeline[-1]['timestamp']}")
    
    return timeline

def analyze_data(sms_data, call_data, email_data, user_phone=None):
    """Enhanced analysis for forensic data"""
    print("\n" + "-"*70)
    print("DATA ANALYSIS")
    print("-"*70)
    
    # Calculate statistics
    total_events = len(sms_data) + len(call_data) + len(email_data)
    
    if total_events == 0:
        print(" No data available for analysis.")
        return []
    
    print(f"\n DATA VOLUME ANALYSIS")
    print(f"Total Forensic Events: {total_events:,}")
    print(f"•  SMS Messages: {len(sms_data):,} ({len(sms_data)/total_events*100:.1f}%)")
    print(f"•  Phone Calls: {len(call_data):,} ({len(call_data)/total_events*100:.1f}%)")
    print(f"•  Emails: {len(email_data):,} ({len(email_data)/total_events*100:.1f}%)")
    
    if user_phone:
        print(f"\n USER IDENTIFICATION")
        print(f"User's Phone Number: {user_phone}")
    
    # Extract contacts
    contact_counts, contact_details = extract_contacts(sms_data, call_data, email_data)
    print(f"\n CONTACT NETWORK ANALYSIS")
    print(f"Unique Contacts Found: {len(contact_counts):,}")
    
    # Show top contacts
    if contact_counts:
        top_contacts = contact_counts.most_common(15)
        print(f"\n TOP 15 MOST ACTIVE CONTACTS")
        print("-" * 80)
        print(f"{'Rank':<5} {'Contact':<35} {'Total':<8} {'SMS':<8} {'Calls':<8} {'Emails':<10}")
        print("-" * 80)
        
        for i, (contact, total_count) in enumerate(top_contacts[:15], 1):
            details = contact_details.get(contact, {})
            sms_count = details.get('sms_count', 0)
            call_count = details.get('call_count', 0)
            email_sent = details.get('sent_email_count', 0)
            email_received = details.get('received_email_count', 0)
            email_total = email_sent + email_received
            
            # Mark user's number if found
            contact_display = contact
            if user_phone and contact == user_phone:
                contact_display = f"📱 {contact} (USER)"
            
            contact_display = contact_display[:32] + "..." if len(contact_display) > 32 else contact_display
            print(f"{i:<5} {contact_display:<35} {total_count:<8} {sms_count:<8} {call_count:<8} {email_total:<10}")
    
    # Create timeline
    timeline = create_timeline(sms_data, call_data, email_data, user_phone)
    
    if timeline:
        # Timeline analysis
        start_date = timeline[0]['timestamp']
        end_date = timeline[-1]['timestamp']
        days_span = max((end_date - start_date).days, 1)
        
        print(f"\n TIMELINE ANALYSIS")
        print(f"Investigation Period: {days_span} days ({start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')})")
        print(f"Average daily events: {total_events/days_span:.1f}")
        print(f"Events per hour: {total_events/(days_span*24):.1f}")
        
        # Forensic categories with reasons
        print(f"\n FORENSIC CATEGORY ANALYSIS")
        categories = Counter(event['forensic_tag'] for event in timeline)
        total_categorized = len(timeline)
        
        print(f"{'Category':<15} {'Count':>10} {'Percentage':>12}")
        print("-" * 40)
        for category, count in categories.most_common():
            percentage = (count / total_categorized) * 100
            print(f"{category:<15} {count:>10,} {percentage:>11.1f}%")
        
        # Show reasons for suspicious communications
        suspicious_events = [e for e in timeline if e['forensic_tag'] in ['SUSPICIOUS', 'FINANCIAL', 'URGENT', 'INTERNATIONAL']]
        if suspicious_events:
            print(f"\n SUSPICIOUS COMMUNICATION REASONS")
            print("-" * 60)
            
            # Count common reasons
            reason_counter = Counter()
            for event in suspicious_events:
                for reason in event.get('reasons', []):
                    reason_counter[reason] += 1
            
            for reason, count in reason_counter.most_common(10):
                print(f"• {reason}: {count:,} occurrences")
        
        # Suspicious patterns with detailed explanations
        print(f"\n DETAILED RISK ASSESSMENT")
        flags = detect_suspicious_patterns_with_details(timeline, user_phone)
        
        if flags:
            risk_score = min(len(flags) * 10, 100)
            print(f"Overall Risk Score: {risk_score}/100")
            print("\n🔴 RED FLAGS DETECTED:")
            for flag, explanation in flags:
                print(f"  ⚠ {flag}")
                print(f"     {explanation}")
        else:
            print("No significant red flags detected.")
            print("Risk Score: 0/100 (Low Risk)")
    
    return timeline

def detect_suspicious_patterns_with_details(timeline, user_phone=None):
    """Detect potentially suspicious patterns with detailed explanations"""
    flags = []
    
    if not timeline or len(timeline) < 10:
        return flags
    
    total_events = len(timeline)
    
    # 1. Late-night communications (midnight to 5 AM)
    late_night = [e for e in timeline if 0 <= e['timestamp'].hour <= 5]
    late_night_percentage = len(late_night) / total_events * 100
    if late_night_percentage > 20:  # More than 20% at night
        flags.append((
            f"High late-night activity: {len(late_night):,} events ({late_night_percentage:.1f}%)",
            f"Normal business/personal communications typically occur during daytime. High nighttime activity ({late_night_percentage:.1f}%) may indicate covert communications."
        ))
    
    # 2. Rapid communications (multiple events within minutes)
    timeline.sort(key=lambda x: x['timestamp'])
    rapid_sequences = 0
    for i in range(1, len(timeline)):
        time_diff = (timeline[i]['timestamp'] - timeline[i-1]['timestamp']).seconds
        if time_diff < 30:  # Less than 30 seconds
            rapid_sequences += 1
    
    if rapid_sequences > total_events * 0.05:  # More than 5%
        flags.append((
            f"Rapid-fire communications: {rapid_sequences:,} sequences <30s apart",
            f"Multiple communications within seconds may indicate coordination, panic, or automated communications."
        ))
    
    # 3. Unknown contacts
    unknown_contacts = sum(1 for e in timeline if 'unknown' in str(e.get('contact', '')).lower())
    if unknown_contacts > total_events * 0.1:  # More than 10%
        flags.append((
            f"High unknown contacts: {unknown_contacts:,} ({unknown_contacts/total_events*100:.1f}%)",
            f"High percentage of communications with unknown/unidentified contacts may indicate attempts to hide identities."
        ))
    
    # 4. Financial-related communications
    financial_events = sum(1 for e in timeline if e.get('forensic_tag') == 'FINANCIAL')
    if financial_events > 10:
        flags.append((
            f"Financial-related communications: {financial_events:,}",
            f"Multiple financial-related communications may indicate money transfers, scams, or financial crimes."
        ))
    
    # 5. Suspicious keyword communications
    suspicious_events = sum(1 for e in timeline if e.get('forensic_tag') == 'SUSPICIOUS')
    if suspicious_events > 5:
        flags.append((
            f"Suspicious keyword communications: {suspicious_events:,}",
            f"Communications contain suspicious keywords like 'delete', 'secret', 'encrypt', etc."
        ))
    
    # 6. International communications
    international_events = sum(1 for e in timeline if e.get('forensic_tag') == 'INTERNATIONAL')
    if international_events > 3:
        flags.append((
            f"International communications: {international_events:,}",
            f"International communications may indicate foreign contacts or cross-border activities."
        ))
    
    # 7. Extended communications
    extended_comms = sum(1 for e in timeline if e.get('forensic_tag') == 'EXTENDED_COMM')
    if extended_comms > 2:
        flags.append((
            f"Extended communications (>1 hour): {extended_comms:,}",
            f"Exceptionally long communications may indicate detailed planning or negotiations."
        ))
    
    # 8. Activity on weekends
    weekend_events = sum(1 for e in timeline if e['timestamp'].weekday() >= 5)  # 5=Sat, 6=Sun
    weekend_percentage = weekend_events / total_events * 100
    if weekend_percentage > 40:  # More than 40% on weekends
        flags.append((
            f"High weekend activity: {weekend_percentage:.1f}%",
            f"Unusually high weekend activity may indicate non-business related communications or shift work."
        ))
    
    # 9. Communications with single contact
    contact_counts = Counter(e.get('contact', 'Unknown') for e in timeline)
    if contact_counts:
        top_contact, top_count = contact_counts.most_common(1)[0]
        if top_count > total_events * 0.3:  # More than 30% with one contact
            flags.append((
                f"High concentration with single contact: {top_contact[:20]}... ({top_count:,} events, {top_count/total_events*100:.1f}%)",
                f"Over 30% of all communications are with a single contact, which may indicate a primary relationship or dependency."
            ))
    
    # 10. Communications during business hours (if user phone is known)
    if user_phone:
        business_hours_events = sum(1 for e in timeline if 9 <= e['timestamp'].hour <= 17)
        business_percentage = business_hours_events / total_events * 100
        if business_percentage < 20:  # Less than 20% during business hours
            flags.append((
                f"Low business hours activity: {business_percentage:.1f}%",
                f"Most communications occur outside normal business hours (9AM-5PM), which may indicate non-work related activities."
            ))
    
    return flags[:10]

def extract_topic_from_content(content, medium):
    """Extract potential topics from communication content"""
    if not content:
        return None
    
    content_lower = str(content).lower()
    
    # Common topics (expand based on your needs)
    topic_keywords = {
        'MEETING': ['meet', 'meeting', 'appointment', 'schedule', 'calendar', 'venue', 'location'],
        'PAYMENT': ['payment', 'money', 'transfer', 'bank', 'cash', 'fund', 'bitcoin', 'crypto'],
        'URGENT': ['urgent', 'emergency', 'asap', 'immediately', 'now', 'quick', 'rush'],
        'SECURITY': ['secure', 'password', 'login', 'access', 'code', 'key', 'encrypt'],
        'BUSINESS': ['project', 'deal', 'contract', 'agreement', 'client', 'customer', 'sale'],
        'PERSONAL': ['family', 'home', 'love', 'dear', 'miss', 'happy', 'birthday'],
        'TRAVEL': ['travel', 'flight', 'ticket', 'hotel', 'reservation', 'trip', 'airport'],
    }
    
    # Try to extract the most prominent topic
    detected_topics = []
    for topic, keywords in topic_keywords.items():
        for keyword in keywords:
            if keyword in content_lower:
                detected_topics.append(topic)
                break
    
    # Also extract specific keywords
    specific_keywords = []
    important_words = ['code', 'password', 'address', 'location', 'time', 'date', 'amount', 'price']
    for word in important_words:
        if word in content_lower:
            specific_keywords.append(word)
    
    if detected_topics or specific_keywords:
        return {
            'main_topics': list(set(detected_topics)),
            'keywords': specific_keywords
        }
    
    return None

def detect_coordinated_communications(timeline, user_phone=None, time_window_minutes=30):
    """
    Detect coordinated communications across multiple channels
    
    Looks for patterns like:
    - SMS → Call → Email to same person within time window
    - Multiple communications to same person across channels
    - Same topic discussed across different media
    """
    print("\n" + "="*70)
    print(" MULTI-CHANNEL COMMUNICATION ANALYSIS")
    print("="*70)
    
    if not timeline:
        print(" No timeline data available.")
        return []
    
    if len(timeline) < 3:
        print(" Insufficient data for pattern detection (need at least 3 events).")
        return []
    
    coordinated_patterns = []
    
    # Group communications by contact
    communications_by_contact = defaultdict(list)
    
    for event in timeline:
        try:
            source, destination, medium, direction, content = get_communication_details(event, user_phone)
            
            # Determine which contact is not the user
            user_id = user_phone if user_phone else "User's Phone"
            contact = destination if source == user_id else source
            
            # Skip if contact is user
            if contact == user_id:
                continue
            
            # Extract topic (with error handling)
            topic_info = None
            try:
                topic_info = extract_topic_from_content(content, medium)
            except:
                pass
            
            communications_by_contact[contact].append({
                'event': event,
                'timestamp': event['timestamp'],
                'medium': medium,
                'direction': 'FROM_USER' if source == user_id else 'TO_USER',
                'content': content[:100] if content else "No content",
                'topic': topic_info
            })
        except Exception as e:
            print(f"  Warning: Error processing event: {e}")
            continue
    
    # Analyze each contact's communication patterns
    for contact, comms in communications_by_contact.items():
        if len(comms) < 3:  # Need at least 3 communications for pattern
            continue
        
        # Sort by timestamp
        comms_sorted = sorted(comms, key=lambda x: x['timestamp'])
        
        # Look for multi-channel sequences using sliding window
        for i in range(len(comms_sorted) - 2):
            try:
                current = comms_sorted[i]
                next1 = comms_sorted[i + 1]
                next2 = comms_sorted[i + 2]
                
                # Check if all within time window (e.g., 30 minutes)
                time_diff1 = (next1['timestamp'] - current['timestamp']).total_seconds() / 60
                time_diff2 = (next2['timestamp'] - next1['timestamp']).total_seconds() / 60
                
                if time_diff1 <= time_window_minutes and time_diff2 <= time_window_minutes:
                    # Check if using different media
                    media_used = {current['medium'], next1['medium'], next2['medium']}
                    
                    if len(media_used) >= 2:  # At least 2 different media
                        # Check for topic consistency
                        topics = []
                        for comm in [current, next1, next2]:
                            if comm['topic'] and comm['topic'].get('main_topics'):
                                topics.extend(comm['topic']['main_topics'])
                        
                        pattern = {
                            'contact': contact,
                            'time_start': current['timestamp'],
                            'time_end': next2['timestamp'],
                            'duration_minutes': (next2['timestamp'] - current['timestamp']).total_seconds() / 60,
                            'sequence': [
                                (current['medium'], current['direction'], current['content'][:50]),
                                (next1['medium'], next1['direction'], next1['content'][:50]),
                                (next2['medium'], next2['direction'], next2['content'][:50])
                            ],
                            'media_used': list(media_used),
                            'common_topics': [],
                            'pattern_type': 'MULTI_CHANNEL_COORDINATED',
                            'risk_level': 'HIGH' if len(media_used) == 3 else 'MEDIUM'
                        }
                        
                        # Find common topics (if any)
                        if topics:
                            topic_counts = Counter(topics)
                            pattern['common_topics'] = [topic for topic, count in topic_counts.items() if count > 1]
                        
                        coordinated_patterns.append(pattern)
            except Exception as e:
                print(f"  Warning: Error analyzing sequence: {e}")
                continue
    
    # Print summary
    if coordinated_patterns:
        high_risk = [p for p in coordinated_patterns if p['risk_level'] == 'HIGH']
        medium_risk = [p for p in coordinated_patterns if p['risk_level'] == 'MEDIUM']
        
        print(f"\n📊 **PATTERN DETECTION SUMMARY:**")
        print(f"   • Total patterns found: {len(coordinated_patterns)}")
        print(f"   • HIGH RISK (3 channels): {len(high_risk)}")
        print(f"   • MEDIUM RISK (2 channels): {len(medium_risk)}")
        
        # Show example of SMS→Call→Email pattern if exists
        sms_call_email = [p for p in high_risk if set(['SMS', 'CALL', 'EMAIL']).issubset(p['media_used'])]
        if sms_call_email:
            print(f"\n⚠️ **EXAMPLE SMS→CALL→EMAIL PATTERN DETECTED:**")
            example = sms_call_email[0]
            print(f"   Contact: {example['contact'][:30]}...")
            print(f"   Time window: {example['duration_minutes']:.1f} minutes")
            print(f"   Sequence:")
            for j, (medium, direction, content) in enumerate(example['sequence'], 1):
                arrow = "→" if j < 3 else ""
                print(f"      {j}. {medium} {arrow}")
    else:
        print("\n✅ No coordinated multi-channel patterns detected.")
    
    return coordinated_patterns

def enhanced_suspicious_analysis(timeline, user_phone=None):
    """Enhanced analysis including multi-channel patterns"""
    # Existing analysis
    suspicious_tags = ['SUSPICIOUS', 'FINANCIAL', 'URGENT', 'INTERNATIONAL', 'EXTENDED_COMM', 'SPAM']
    suspicious_events = [e for e in timeline if e['forensic_tag'] in suspicious_tags]
    
    # NEW: Detect coordinated multi-channel communications
    coordinated_patterns = detect_coordinated_communications(timeline, user_phone)
    
    print(f"\n🔍 COORDINATED MULTI-CHANNEL PATTERNS DETECTED: {len(coordinated_patterns):,}")
    
    if coordinated_patterns:
        print("\n" + "="*70)
        print(" DETAILED COORDINATED COMMUNICATION PATTERNS")
        print("="*70)
        
        for i, pattern in enumerate(coordinated_patterns[:10], 1):  # Show top 10
            print(f"\n{i}. 🚨 **COORDINATED PATTERN WITH {pattern['contact'][:30]}...**")
            print(f"   Time: {pattern['time_start'].strftime('%Y-%m-%d %H:%M')} → {pattern['time_end'].strftime('%H:%M')}")
            print(f"   Duration: {pattern['duration_minutes']:.1f} minutes")
            print(f"   Media Used: {', '.join(pattern['media_used'])}")
            print(f"   Risk Level: {pattern['risk_level']}")
            
            if pattern['common_topics']:
                print(f"   Common Topics: {', '.join(pattern['common_topics'])}")
            
            print(f"   Communication Sequence:")
            for j, (medium, direction, content) in enumerate(pattern['sequence'], 1):
                dir_symbol = "📤" if direction == 'FROM_USER' else "📥"
                print(f"     {j}. {dir_symbol} {medium}: {content}...")
            
            # Add forensic interpretation
            print(f"\n   🔍 **FORENSIC INTERPRETATION:**")
            if len(pattern['media_used']) == 3:
                print(f"     • SMS → Call → Email pattern detected")
                print(f"     • **HIGH RISK**: Coordinated communication across all major channels")
                print(f"     • Could indicate: Urgent coordination, crisis management, or covert planning")
            else:
                print(f"     • Multi-channel communication pattern detected")
                print(f"     • Could indicate: Detailed discussion, confirmation seeking, or layered communication")
    
    return coordinated_patterns

# ============================================================================
# ENHANCED FUNCTIONS FOR CLEAR SOURCE/DESTINATION DISPLAY
# ============================================================================

def get_communication_details(event, user_phone):
    """Get clear source and destination details from event"""
    source_type = event.get('source', 'UNKNOWN')
    contact = event.get('contact', 'Unknown')
    
    if source_type == 'SMS':
        direction = event.get('type', '').upper()
        if direction == 'OUTGOING':
            # Outgoing SMS: User sent to contact
            source = user_phone if user_phone else "User's Phone"
            return source, contact, 'SMS', direction, event.get('content', '')
        elif direction == 'INCOMING':
            # Incoming SMS: Contact sent to user
            return contact, user_phone if user_phone else "User's Phone", 'SMS', direction, event.get('content', '')
        else:
            return contact, user_phone if user_phone else "User's Phone", 'SMS', 'UNKNOWN', event.get('content', '')
    
    elif source_type == 'CALL':
        call_type = event.get('type', 'UNKNOWN')
        # For calls: contact called user
        # CDR typically shows the caller's number
        source = contact
        destination = user_phone if user_phone else "User's Phone"
        return source, destination, 'CALL', call_type, f"Duration: {event.get('details', {}).get('duration', 0)}s"
    
    elif source_type == 'EMAIL':
        sender = contact  # In timeline, contact is sender for emails
        recipient = event.get('details', {}).get('recipient', 'Unknown')
        subject = event.get('content', '').split('Subject:')[1].strip() if 'Subject:' in event.get('content', '') else 'No Subject'
        
        # For emails, just show sender and recipient as is
        return sender, recipient, 'EMAIL', 'SENT', subject
    
    return contact, user_phone if user_phone else "User's Phone", source_type, 'UNKNOWN', ''

def display_timeline_events(timeline, user_phone=None):
    """Display all timeline events with clear source/destination"""
    print("\n" + "-"*120)
    print(" TIMELINE EVENTS - ALL COMMUNICATIONS")
    print("-"*120)
    
    if not timeline:
        print("No timeline events available.")
        return
    
    total_events = len(timeline)
    print(f"\n📊 TOTAL COMMUNICATION EVENTS: {total_events:,}")
    
    if user_phone:
        print(f"📱 USER'S PHONE NUMBER: {user_phone}")
    
    # Group by communication type
    print("\n" + "-"*120)
    print(" 1. SMS MESSAGES")
    print("-"*120)
    sms_events = [e for e in timeline if e['source'] == 'SMS']
    if sms_events:
        print(f"\nTotal SMS Messages: {len(sms_events):,}")
        print("\n" + "-" * 120)
        print(f"{'Time':<20} {'From':<25} {'→':<3} {'To':<25} {'Direction':<10} {'Category':<15} {'Message':<30}")
        print("-" * 120)
        
        for event in sms_events[:50]:  # Show first 50 SMS
            timestamp = event['timestamp'].strftime('%Y-%m-%d %H:%M')
            source, destination, medium, direction, content = get_communication_details(event, user_phone)
            
            # Truncate long strings
            source_display = source[:23] + "..." if len(source) > 23 else source
            dest_display = destination[:23] + "..." if len(destination) > 23 else destination
            message_preview = content[:28] + "..." if len(content) > 28 else content
            
            # Show actual direction
            actual_direction = "SENT" if direction == "OUTGOING" else "RECEIVED"
            
            # Show forensic category
            category = event.get('forensic_tag', 'ROUTINE')
            
            print(f"{timestamp:<20} {source_display:<25} {'→':<3} {dest_display:<25} {actual_direction:<10} {category:<15} {message_preview:<30}")
        
        if len(sms_events) > 50:
            print(f"... and {len(sms_events) - 50:,} more SMS messages")
        
        # SMS Statistics
        incoming_sms = [e for e in sms_events if e.get('type') == 'INCOMING']
        outgoing_sms = [e for e in sms_events if e.get('type') == 'OUTGOING']
        print(f"\n📱 SMS STATISTICS:")
        print(f"   • Received SMS: {len(incoming_sms):,} ({len(incoming_sms)/len(sms_events)*100:.1f}%)")
        print(f"   • Sent SMS: {len(outgoing_sms):,} ({len(outgoing_sms)/len(sms_events)*100:.1f}%)")
        
    else:
        print("No SMS messages found.")
    
    print("\n" + "-"*120)
    print(" 2. PHONE CALLS")
    print("-"*120)
    call_events = [e for e in timeline if e['source'] == 'CALL']
    if call_events:
        print(f"\nTotal Phone Calls: {len(call_events):,}")
        print("\n" + "-" * 120)
        print(f"{'Time':<20} {'Caller':<25} {'→':<3} {'Receiver':<25} {'Call Type':<15} {'Duration':<10} {'Category':<15}")
        print("-" * 120)
        
        for event in call_events[:50]:  # Show first 50 calls
            timestamp = event['timestamp'].strftime('%Y-%m-%d %H:%M')
            source, destination, medium, call_type, details = get_communication_details(event, user_phone)
            
            # Get duration
            duration = event.get('details', {}).get('duration', 0)
            duration_str = f"{duration//60}:{duration%60:02d}" if duration > 60 else f"{duration}s"
            
            # Truncate
            source_display = source[:23] + "..." if len(source) > 23 else source
            dest_display = destination[:23] + "..." if len(destination) > 23 else destination
            
            # Show forensic category
            category = event.get('forensic_tag', 'ROUTINE')
            
            print(f"{timestamp:<20} {source_display:<25} {'→':<3} {dest_display:<25} {call_type:<15} {duration_str:<10} {category:<15}")
        
        if len(call_events) > 50:
            print(f"... and {len(call_events) - 50:,} more phone calls")
        
        # Call Statistics
        total_duration = sum(e.get('details', {}).get('duration', 0) for e in call_events)
        avg_duration = total_duration / len(call_events) if call_events else 0
        
        call_types = Counter(e.get('type', 'UNKNOWN') for e in call_events)
        print(f"\n📞 CALL STATISTICS:")
        print(f"   • Total call duration: {total_duration//3600}h {(total_duration%3600)//60}m {total_duration%60}s")
        print(f"   • Average call duration: {avg_duration:.1f} seconds")
        print(f"   • Call types:")
        for call_type, count in call_types.most_common():
            print(f"      - {call_type}: {count:,}")
        
    else:
        print("No phone calls found.")
    
    print("\n" + "-"*120)
    print(" 3. EMAILS")
    print("-"*120)
    email_events = [e for e in timeline if e['source'] == 'EMAIL']
    if email_events:
        print(f"\nTotal Emails: {len(email_events):,}")
        print("\n" + "-" * 120)
        print(f"{'Time':<20} {'From':<30} {'→':<3} {'To':<30} {'Category':<15} {'Subject':<30}")
        print("-" * 120)
        
        for event in email_events[:50]:  # Show first 50 emails
            timestamp = event['timestamp'].strftime('%Y-%m-%d %H:%M')
            source, destination, medium, _, subject = get_communication_details(event, user_phone)
            
            subject_display = (subject[:28] + "...") if len(subject) > 28 else subject
            
            # Truncate
            source_display = source[:28] + "..." if len(source) > 28 else source
            dest_display = destination[:28] + "..." if len(destination) > 28 else destination
            
            # Show forensic category
            category = event.get('forensic_tag', 'ROUTINE')
            
            print(f"{timestamp:<20} {source_display:<30} {'→':<3} {dest_display:<30} {category:<15} {subject_display:<30}")
        
        if len(email_events) > 50:
            print(f"... and {len(email_events) - 50:,} more emails")
        
        # Email Statistics
        unique_senders = set()
        unique_recipients = set()
        for event in email_events:
            source, destination, _, _, _ = get_communication_details(event, user_phone)
            unique_senders.add(source)
            unique_recipients.add(destination)
        
        print(f"\n📧 EMAIL STATISTICS:")
        print(f"   • Unique senders: {len(unique_senders):,}")
        print(f"   • Unique recipients: {len(unique_recipients):,}")
        
    else:
        print("No emails found.")
    
    # Summary statistics
    print("\n" + "-"*120)
    print(" COMMUNICATION SUMMARY")
    print("-"*120)
    print(f"\n📈 TOTAL EVENTS: {total_events:,}")
    print(f"   • SMS Messages: {len(sms_events):,} ({len(sms_events)/total_events*100:.1f}%)")
    print(f"   • Phone Calls: {len(call_events):,} ({len(call_events)/total_events*100:.1f}%)")
    print(f"   • Emails: {len(email_events):,} ({len(email_events)/total_events*100:.1f}%)")
    
    # Time range
    if timeline:
        earliest = timeline[0]['timestamp']
        latest = timeline[-1]['timestamp']
        days_span = (latest - earliest).days + 1
        print(f"\n📅 TIME RANGE: {earliest.strftime('%Y-%m-%d')} to {latest.strftime('%Y-%m-%d')}")
        print(f"   • Duration: {days_span} days")
        print(f"   • Average daily events: {total_events/days_span:.1f}")
        
        # Most active day
        daily_counts = {}
        for event in timeline:
            date = event['timestamp'].date()
            daily_counts[date] = daily_counts.get(date, 0) + 1
        
        if daily_counts:
            busiest_date = max(daily_counts, key=daily_counts.get)
            print(f"   • Busiest day: {busiest_date} ({daily_counts[busiest_date]:,} events)")

def view_detailed_analysis(timeline, sms_data, call_data, email_data, user_phone=None):
    """Display detailed analysis with source/destination information"""
    print("\n" + "-"*120)
    print(" DETAILED COMMUNICATION ANALYSIS")
    print("-"*120)
    
    if not timeline:
        print("No timeline data available.")
        return
    
    total_events = len(timeline)
    print(f"\n📊 TOTAL COMMUNICATION EVENTS: {total_events:,}")
    
    if user_phone:
        print(f"📱 USER'S PHONE NUMBER: {user_phone}")
    
    # 1. Communication Flow Analysis
    print("\n" + "-"*120)
    print(" 1. COMMUNICATION FLOW ANALYSIS")
    print("="*120)
    
    # Count communications by type with source/destination
    communication_flows = defaultdict(int)
    
    for event in timeline:
        source, destination, medium, direction, _ = get_communication_details(event, user_phone)
        flow_key = f"{source} → {destination}"
        communication_flows[flow_key] += 1
    
    # Show top communication flows
    print("\nTOP COMMUNICATION FLOWS:")
    print("-" * 100)
    print(f"{'Rank':<5} {'From → To':<50} {'Count':<10} {'%':<8}")
    print("-" * 100)
    
    sorted_flows = sorted(communication_flows.items(), key=lambda x: x[1], reverse=True)
    for i, (flow, count) in enumerate(sorted_flows[:20], 1):
        percentage = (count / total_events) * 100
        flow_display = flow[:48] + "..." if len(flow) > 48 else flow
        print(f"{i:<5} {flow_display:<50} {count:<10,} {percentage:>7.1f}%")
    
    # 2. Contact Analysis
    print("\n" + "-"*120)
    print(" 2. CONTACT ANALYSIS")
    print("-"*120)
    
    contact_counts, contact_details = extract_contacts(sms_data, call_data, email_data)
    print(f"\n👥 UNIQUE CONTACTS: {len(contact_counts):,}")
    
    # Show incoming communications (to user)
    incoming_communications = Counter()
    # Show outgoing communications (from user)
    outgoing_communications = Counter()
    
    for event in timeline:
        source, destination, medium, direction, _ = get_communication_details(event, user_phone)
        
        # Determine if this is incoming or outgoing relative to user
        user_identifier = user_phone if user_phone else "User's Phone"
        
        if destination == user_identifier and source != user_identifier:
            # Incoming communication to user
            incoming_communications[source] += 1
        
        if source == user_identifier and destination != user_identifier:
            # Outgoing communication from user
            outgoing_communications[destination] += 1
    
    if incoming_communications:
        print(f"\n📥 COMMUNICATIONS RECEIVED BY USER:")
        print("-" * 80)
        print(f"{'Rank':<5} {'From':<40} {'Count':<10} {'%':<8}")
        print("-" * 80)
        
        total_incoming = sum(incoming_communications.values())
        for i, (contact, count) in enumerate(incoming_communications.most_common(15), 1):
            percentage = (count / total_incoming) * 100
            contact_display = contact[:38] + "..." if len(contact) > 38 else contact
            print(f"{i:<5} {contact_display:<40} {count:<10,} {percentage:>7.1f}%")
    
    if outgoing_communications:
        print(f"\n📤 COMMUNICATIONS SENT BY USER:")
        print("-" * 80)
        print(f"{'Rank':<5} {'To':<40} {'Count':<10} {'%':<8}")
        print("-" * 80)
        
        total_outgoing = sum(outgoing_communications.values())
        for i, (contact, count) in enumerate(outgoing_communications.most_common(15), 1):
            percentage = (count / total_outgoing) * 100
            contact_display = contact[:38] + "..." if len(contact) > 38 else contact
            print(f"{i:<5} {contact_display:<40} {count:<10,} {percentage:>7.1f}%")
    
    # 3. Communication Pattern Analysis
    print("\n" + "-"*120)
    print(" 3. COMMUNICATION PATTERNS")
    print("-"*120)
    
    # Analyze communication patterns by type
    communication_patterns = defaultdict(lambda: defaultdict(int))
    
    for event in timeline:
        source, destination, medium, direction, _ = get_communication_details(event, user_phone)
        pattern_key = f"{source} ↔ {destination}"
        communication_patterns[pattern_key][medium] += 1
    
    print(f"\nTOP COMMUNICATION PATTERNS BY MEDIUM:")
    print("-" * 100)
    print(f"{'Rank':<5} {'Parties':<40} {'SMS':<8} {'Calls':<8} {'Emails':<8} {'Total':<10}")
    print("-" * 100)
    
    # Calculate total communications per pattern
    pattern_totals = []
    for pattern, mediums in communication_patterns.items():
        total = sum(mediums.values())
        pattern_totals.append((pattern, total, mediums))
    
    # Sort by total communications
    pattern_totals.sort(key=lambda x: x[1], reverse=True)
    
    for i, (pattern, total, mediums) in enumerate(pattern_totals[:15], 1):
        pattern_display = pattern[:38] + "..." if len(pattern) > 38 else pattern
        print(f"{i:<5} {pattern_display:<40} "
              f"{mediums.get('SMS', 0):<8} "
              f"{mediums.get('CALL', 0):<8} "
              f"{mediums.get('EMAIL', 0):<8} "
              f"{total:<10,}")
    
    # 4. Temporal Analysis
    print("\n" + "-"*120)
    print(" 4. TEMPORAL ANALYSIS")
    print("-"*120)
    
    timeline_sorted = sorted(timeline, key=lambda x: x['timestamp'])
    if timeline_sorted:
        earliest = timeline_sorted[0]['timestamp']
        latest = timeline_sorted[-1]['timestamp']
        days_span = (latest - earliest).days + 1
        
        print(f"\n📅 INVESTIGATION PERIOD: {days_span} days")
        print(f"   • From: {earliest.strftime('%Y-%m-%d %H:%M')}")
        print(f"   • To: {latest.strftime('%Y-%m-%d %H:%M')}")
        print(f"   • Average daily communications: {total_events/days_span:.1f}")
        
        # Busiest hours
        hourly_counts = {hour: 0 for hour in range(24)}
        for event in timeline:
            hour = event['timestamp'].hour
            hourly_counts[hour] += 1
        
        peak_hour = max(hourly_counts, key=hourly_counts.get)
        print(f"\n⏰ PEAK ACTIVITY HOUR: {peak_hour:02d}:00 ({hourly_counts[peak_hour]:,} events)")
        
        # Communication by time of day
        morning = sum(hourly_counts[h] for h in range(6, 12))
        afternoon = sum(hourly_counts[h] for h in range(12, 18))
        evening = sum(hourly_counts[h] for h in range(18, 24))
        night = sum(hourly_counts[h] for h in range(0, 6))
        
        print(f"\n🌅 COMMUNICATION BY TIME OF DAY:")
        print(f"   • Morning (6AM-12PM): {morning:,} ({morning/total_events*100:.1f}%)")
        print(f"   • Afternoon (12PM-6PM): {afternoon:,} ({afternoon/total_events*100:.1f}%)")
        print(f"   • Evening (6PM-12AM): {evening:,} ({evening/total_events*100:.1f}%)")
        print(f"   • Night (12AM-6AM): {night:,} ({night/total_events*100:.1f}%)")
    
    # 5. Forensic Category Analysis with Reasons
    print("\n" + "="*120)
    print(" 5. FORENSIC CATEGORY ANALYSIS WITH REASONS")
    print("="*120)
    
    # Group events by category and collect reasons
    category_events = defaultdict(list)
    for event in timeline:
        category = event.get('forensic_tag', 'ROUTINE')
        category_events[category].append(event)
    
    # Show categories with example reasons
    suspicious_categories = ['SUSPICIOUS', 'FINANCIAL', 'URGENT', 'INTERNATIONAL', 'EXTENDED_COMM', 'SPAM']
    
    for category in suspicious_categories:
        if category_events[category]:
            events = category_events[category]
            print(f"\n🔍 {category} COMMUNICATIONS: {len(events):,} events")
            
            # Show top reasons for this category
            reason_counter = Counter()
            for event in events[:100]:  # Check first 100 events for reasons
                for reason in event.get('reasons', []):
                    reason_counter[reason] += 1
            
            if reason_counter:
                print(f"   Top reasons for {category} classification:")
                for reason, count in reason_counter.most_common(5):
                    print(f"      • {reason}: {count:,} occurrences")
            
            # Show example events
            if len(events) > 0:
                print(f"   Example {category.lower()} communication:")
                example = events[0]
                timestamp = example['timestamp'].strftime('%Y-%m-%d %H:%M')
                source, destination, medium, _, content = get_communication_details(example, user_phone)
                print(f"      Time: {timestamp}")
                print(f"      From: {source}")
                print(f"      To: {destination}")
                print(f"      Content: {content[:100]}...")
                if example.get('reasons'):
                    print(f"      Reasons: {', '.join(example['reasons'][:3])}")

def suspicious_communications_analysis(timeline, user_phone=None):
    """Analyze and display suspicious communications with detailed reasons"""
    print("\n" + "="*120)
    print(" SUSPICIOUS COMMUNICATIONS ANALYSIS")
    print("="*120)
    
    if not timeline:
        print("No timeline data available.")
        return
    
    if user_phone:
        print(f"📱 USER'S PHONE NUMBER: {user_phone}")
    
    # Get suspicious events based on forensic tags
    suspicious_tags = ['SUSPICIOUS', 'FINANCIAL', 'URGENT', 'INTERNATIONAL', 'EXTENDED_COMM', 'SPAM']
    suspicious_events = [e for e in timeline if e['forensic_tag'] in suspicious_tags]
    
    total_events = len(timeline)
    suspicious_count = len(suspicious_events)
    
    print(f"\n🔍 SUSPICIOUS COMMUNICATIONS DETECTED: {suspicious_count:,} of {total_events:,} total events")
    print(f"   • Suspicious Rate: {suspicious_count/total_events*100:.1f}%")
    
    if suspicious_events:
        # Group by suspicious category with detailed breakdown
        print("\n" + "-"*120)
        print(" DETAILED SUSPICIOUS COMMUNICATIONS BREAKDOWN")
        print("-"*120)
        
        # Create detailed breakdown by category
        category_details = defaultdict(lambda: {
            'count': 0,
            'reasons': Counter(),
            'contacts': Counter(),
            'times': []
        })
        
        for event in suspicious_events:
            category = event['forensic_tag']
            details = category_details[category]
            details['count'] += 1
            
            # Count reasons
            for reason in event.get('reasons', []):
                details['reasons'][reason] += 1
            
            # Count contacts
            contact = event.get('contact', 'Unknown')
            details['contacts'][contact] += 1
            
            # Record time
            details['times'].append(event['timestamp'])
        
        # Display detailed breakdown
        print(f"\n{'Category':<20} {'Count':>10} {'% of Suspicious':>15} {'Top Reason':<40} {'Risk':<10}")
        print("-" * 95)
        
        for category, details in sorted(category_details.items(), key=lambda x: x[1]['count'], reverse=True):
            count = details['count']
            percentage = (count / suspicious_count) * 100
            
            # Get top reason
            top_reason = "Unknown"
            if details['reasons']:
                top_reason, reason_count = details['reasons'].most_common(1)[0]
                top_reason = f"{top_reason} ({reason_count}x)"
            
            # Determine risk level
            if category in ['SUSPICIOUS', 'FINANCIAL']:
                risk = 'HIGH'
            elif category in ['URGENT', 'INTERNATIONAL']:
                risk = 'MEDIUM'
            else:
                risk = 'LOW'
            
            print(f"{category:<20} {count:>10,} {percentage:>14.1f}% {top_reason:<40} {risk:<10}")
        
        # Show detailed suspicious communications with reasons
        print("\n" + "-"*120)
        print(" DETAILED SUSPICIOUS COMMUNICATIONS WITH REASONS")
        print("-"*120)
        
        print(f"\nShowing {min(15, len(suspicious_events))} of {len(suspicious_events):,} suspicious communications:")
        print("\n" + "-" * 140)
        print(f"{'No.':<4} {'Time':<18} {'From':<25} {'→':<3} {'To':<25} {'Medium':<8} {'Category':<15} {'Reasons':<60}")
        print("-" * 140)
        
        for i, event in enumerate(suspicious_events[:15], 1):
            timestamp = event['timestamp'].strftime('%m-%d %H:%M')
            source, destination, medium, _, content = get_communication_details(event, user_phone)
            
            # Truncate
            source_display = source[:23] + "..." if len(source) > 23 else source
            dest_display = destination[:23] + "..." if len(destination) > 23 else destination
            
            # Get reasons (truncate if too many)
            reasons = event.get('reasons', [])
            if len(reasons) > 2:
                reasons_display = ', '.join(reasons[:2]) + f" (+{len(reasons)-2} more)"
            else:
                reasons_display = ', '.join(reasons)
            
            reasons_display = reasons_display[:58] + "..." if len(reasons_display) > 58 else reasons_display
            
            print(f"{i:<4} {timestamp:<18} {source_display:<25} {'→':<3} {dest_display:<25} {medium:<8} {event['forensic_tag']:<15} {reasons_display:<60}")
        
        if len(suspicious_events) > 15:
            print(f"\n... and {len(suspicious_events) - 15:,} more suspicious communications")
        
        # Show most suspicious contacts
        print("\n" + "-"*120)
        print(" MOST SUSPICIOUS CONTACTS")
        print("-"*120)
        
        # Count suspicious communications by contact
        contact_suspicion = Counter()
        contact_reasons = defaultdict(set)
        
        for event in suspicious_events:
            source, destination, _, _, _ = get_communication_details(event, user_phone)
            
            # Don't count user
            user_identifier = user_phone if user_phone else "User's Phone"
            if source != user_identifier:
                contact_suspicion[source] += 1
                for reason in event.get('reasons', []):
                    contact_reasons[source].add(reason)
            
            if destination != user_identifier:
                contact_suspicion[destination] += 1
                for reason in event.get('reasons', []):
                    contact_reasons[destination].add(reason)
        
        if contact_suspicion:
            print(f"\nContacts with suspicious communications:")
            print("-" * 100)
            print(f"{'Rank':<5} {'Contact':<40} {'Suspicious Count':<20} {'Example Reasons':<40}")
            print("-" * 100)
            
            for i, (contact, count) in enumerate(contact_suspicion.most_common(10), 1):
                contact_display = contact[:38] + "..." if len(contact) > 38 else contact
                
                # Get example reasons
                reasons = list(contact_reasons.get(contact, set()))
                if len(reasons) > 3:
                    reasons_display = ', '.join(reasons[:3]) + f" (+{len(reasons)-3} more)"
                else:
                    reasons_display = ', '.join(reasons)
                
                reasons_display = reasons_display[:38] + "..." if len(reasons_display) > 38 else reasons_display
                
                print(f"{i:<5} {contact_display:<40} {count:<20,} {reasons_display:<40}")
        
        # Risk Assessment with detailed scoring
        print("\n" + "-"*120)
        print(" DETAILED RISK ASSESSMENT")
        print("-"*120)
        
        # Calculate risk score based on multiple factors
        risk_factors = []
        total_risk_score = 0
        max_risk_score = 0
        
        # Factor 1: Number of suspicious events
        suspicious_ratio = suspicious_count / total_events
        if suspicious_ratio > 0.3:
            factor_score = 30
            risk_factors.append(("High volume of suspicious communications (>{:.1f}%)".format(suspicious_ratio*100), factor_score))
        elif suspicious_ratio > 0.1:
            factor_score = 20
            risk_factors.append(("Moderate volume of suspicious communications (>{:.1f}%)".format(suspicious_ratio*100), factor_score))
        elif suspicious_ratio > 0.05:
            factor_score = 10
            risk_factors.append(("Some suspicious communications (>{:.1f}%)".format(suspicious_ratio*100), factor_score))
        else:
            factor_score = 5
            risk_factors.append(("Low volume of suspicious communications", factor_score))
        
        total_risk_score += factor_score
        max_risk_score += 30
        
        # Factor 2: High-risk categories
        high_risk_events = sum(1 for e in suspicious_events if e['forensic_tag'] in ['SUSPICIOUS', 'FINANCIAL'])
        if high_risk_events > 10:
            factor_score = 25
            risk_factors.append((f"Multiple high-risk communications ({high_risk_events})", factor_score))
        elif high_risk_events > 5:
            factor_score = 15
            risk_factors.append((f"Several high-risk communications ({high_risk_events})", factor_score))
        elif high_risk_events > 0:
            factor_score = 5
            risk_factors.append((f"Some high-risk communications ({high_risk_events})", factor_score))
        else:
            factor_score = 0
            risk_factors.append(("No high-risk communications detected", factor_score))
        
        total_risk_score += factor_score
        max_risk_score += 25
        
        # Factor 3: Time patterns
        late_night = sum(1 for e in suspicious_events if 0 <= e['timestamp'].hour <= 5)
        late_night_ratio = late_night / suspicious_count if suspicious_count > 0 else 0
        if late_night_ratio > 0.3:
            factor_score = 20
            risk_factors.append((f"High late-night suspicious activity ({late_night_ratio*100:.1f}%)", factor_score))
        elif late_night_ratio > 0.1:
            factor_score = 10
            risk_factors.append((f"Moderate late-night suspicious activity ({late_night_ratio*100:.1f}%)", factor_score))
        else:
            factor_score = 0
            risk_factors.append(("Normal time distribution for suspicious communications", factor_score))
        
        total_risk_score += factor_score
        max_risk_score += 20
        
        # Factor 4: Contact concentration
        if contact_suspicion:
            top_contact_count = contact_suspicion.most_common(1)[0][1]
            concentration_ratio = top_contact_count / suspicious_count
            if concentration_ratio > 0.5:
                factor_score = 25
                risk_factors.append((f"High concentration with single contact ({concentration_ratio*100:.1f}%)", factor_score))
            elif concentration_ratio > 0.3:
                factor_score = 15
                risk_factors.append((f"Moderate concentration with contacts ({concentration_ratio*100:.1f}%)", factor_score))
            else:
                factor_score = 5
                risk_factors.append(("Diverse suspicious contacts", factor_score))
        else:
            factor_score = 0
            risk_factors.append(("No suspicious contacts identified", factor_score))
        
        total_risk_score += factor_score
        max_risk_score += 25
        
        # Calculate final risk score
        final_risk_score = min((total_risk_score / max_risk_score) * 100, 100) if max_risk_score > 0 else 0
        
        print(f"\n📊 OVERALL RISK SCORE: {final_risk_score:.1f}/100")
        print("\nRisk Factors Assessment:")
        print("-" * 80)
        print(f"{'Factor':<50} {'Score':<10} {'Max':<10}")
        print("-" * 80)
        
        for factor, score in risk_factors:
            max_for_factor = 30 if "volume" in factor else 25 if "high-risk" in factor else 20 if "late-night" in factor else 25
            print(f"{factor:<50} {score:<10} {max_for_factor:<10}")
        
        print("-" * 80)
        print(f"{'TOTAL':<50} {total_risk_score:<10} {max_risk_score:<10}")
        
        print(f"\n🔍 RISK LEVEL ANALYSIS:")
        if final_risk_score < 20:
            print("   • Risk Level: 🟢 LOW - Minimal suspicious activity")
            print("   • Forensic Priority: Low")
            print("   • Recommendation: Routine monitoring sufficient")
        elif final_risk_score < 50:
            print("   • Risk Level: 🟡 MODERATE - Some concerning patterns")
            print("   • Forensic Priority: Medium")
            print("   • Recommendation: Increased monitoring recommended")
        elif final_risk_score < 75:
            print("   • Risk Level: 🟠 HIGH - Significant suspicious activity")
            print("   • Forensic Priority: High")
            print("   • Recommendation: Immediate investigation recommended")
        else:
            print("   • Risk Level: 🔴 CRITICAL - Extensive suspicious activity")
            print("   • Forensic Priority: Critical")
            print("   • Recommendation: Urgent investigation required")
        
        # Additional suspicious patterns
        detailed_flags = detect_suspicious_patterns_with_details(timeline, user_phone)
        if detailed_flags:
            print(f"\n🚨 ADDITIONAL ANOMALIES DETECTED ({len(detailed_flags)}):")
            for flag, explanation in detailed_flags[:5]:
                print(f"   ⚠ {flag}")
            if len(detailed_flags) > 5:
                print(f"   • ... and {len(detailed_flags) - 5} more anomalies")
    else:
        print("\n✅ No suspicious communications detected.")
        print("   • Risk Level: 🟢 LOW")
        print("   • Forensic Priority: Low")
        print("   • Recommendation: Normal monitoring sufficient")

    print("")
    
    coordinated_patterns = detect_coordinated_communications(timeline, user_phone)
    
    if coordinated_patterns:
        print(f"\n🚨 **CRITICAL FINDING:** {len(coordinated_patterns):,} coordinated multi-channel patterns detected!")
        
        # Count patterns by type
        sms_call_email = sum(1 for p in coordinated_patterns if set(['SMS', 'CALL', 'EMAIL']).issubset(p['media_used']))
        two_channel = len(coordinated_patterns) - sms_call_email
        
        print(f"📊 **Pattern Distribution:**")
        print(f"   🔴 Full coordination (SMS→Call→Email): {sms_call_email:,}")
        print(f"   🟡 Partial coordination (2 channels): {two_channel:,}")
        
        # Update risk score based on coordinated patterns
        if sms_call_email > 0:
            print(f"\n⚠️ **RISK ELEVATION:** Full 3-channel coordination increases overall risk score by +20 points")
            # Adjust your risk scoring logic accordingly
    else:
        print(f"\n✅ No coordinated multi-channel patterns detected.")

# ============================================================================
# EXPORT FUNCTIONS
# ============================================================================

def export_forensic_report(timeline, sms_data, call_data, email_data, user_phone=None):
    """Export comprehensive forensic report"""
    print("\n" + "-"*70)
    print(" EXPORTING THE REPORT")
    print("-"*70)
    
    if not timeline:
        print(" No data to export.")
        return
    
    report_content = []
    report_content.append("-" * 80)
    report_content.append("DIGITAL INVESTIGATION REPORT")
    report_content.append("-" * 80)
    report_content.append(f"Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report_content.append(f"Case Reference: DF-{datetime.now().strftime('%Y%m%d-%H%M%S')}")
    
    if user_phone:
        report_content.append(f"User's Phone Number: {user_phone}")
    
    report_content.append("")
    
    # Executive Summary
    report_content.append("EXECUTIVE SUMMARY")
    report_content.append("-" * 40)
    total_events = len(sms_data) + len(call_data) + len(email_data)
    report_content.append(f"Total Events Analyzed: {total_events:,}")
    
    if timeline:
        days_span = max((timeline[-1]['timestamp'] - timeline[0]['timestamp']).days, 1)
        report_content.append(f"Analysis Period: {days_span} days")
        report_content.append(f"Date Range: {timeline[0]['timestamp'].strftime('%Y-%m-%d')} to {timeline[-1]['timestamp'].strftime('%Y-%m-%d')}")
        report_content.append(f"Average Daily Events: {total_events/days_span:.1f}")
    
    # Data Sources
    report_content.append("")
    report_content.append("DATA SOURCES")
    report_content.append("-" * 40)
    report_content.append(f"SMS Messages: {len(sms_data):,}")
    report_content.append(f"Phone Calls: {len(call_data):,}")
    report_content.append(f"Emails: {len(email_data):,}")
    
    # Contact Analysis
    contact_counts, contact_details = extract_contacts(sms_data, call_data, email_data)
    report_content.append("")
    report_content.append("CONTACT ANALYSIS")
    report_content.append("-" * 40)
    report_content.append(f"Unique Contacts Identified: {len(contact_counts):,}")
    
    if contact_counts:
        top_contacts = contact_counts.most_common(20)
        report_content.append("")
        report_content.append("TOP 20 CONTACTS BY INTERACTION VOLUME:")
        report_content.append("-" * 80)
        report_content.append(f"{'Rank':<5} {'Contact':<40} {'Total':<8} {'SMS':<8} {'Calls':<8} {'Emails':<10}")
        report_content.append("-" * 80)
        
        for i, (contact, total_count) in enumerate(top_contacts, 1):
            details = contact_details.get(contact, {})
            sms_count = details.get('sms_count', 0)
            call_count = details.get('call_count', 0)
            email_sent = details.get('sent_email_count', 0)
            email_received = details.get('received_email_count', 0)
            email_total = email_sent + email_received
            
            # Mark user's number
            contact_display = contact
            if user_phone and contact == user_phone:
                contact_display = f"{contact} (USER)"
            
            contact_display = contact_display[:38] + "..." if len(contact_display) > 38 else contact_display
            report_content.append(f"{i:<5} {contact_display:<40} {total_count:<8} {sms_count:<8} {call_count:<8} {email_total:<10}")
    
    # Suspicious Communications Analysis
    suspicious_tags = ['SUSPICIOUS', 'FINANCIAL', 'URGENT', 'INTERNATIONAL', 'EXTENDED_COMM', 'SPAM']
    suspicious_events = [e for e in timeline if e['forensic_tag'] in suspicious_tags]
    
    if suspicious_events:
        report_content.append("")
        report_content.append("SUSPICIOUS COMMUNICATIONS ANALYSIS")
        report_content.append("-" * 40)
        report_content.append(f"Total Suspicious Communications: {len(suspicious_events):,}")
        report_content.append(f"Suspicious Rate: {len(suspicious_events)/len(timeline)*100:.1f}%")
        
        # Group by category
        category_counts = Counter(e['forensic_tag'] for e in suspicious_events)
        report_content.append("")
        report_content.append("SUSPICIOUS COMMUNICATIONS BY CATEGORY:")
        for category, count in category_counts.most_common():
            report_content.append(f"  • {category}: {count:,} events")
        
        # Top reasons
        reason_counter = Counter()
        for event in suspicious_events:
            for reason in event.get('reasons', []):
                reason_counter[reason] += 1
        
        if reason_counter:
            report_content.append("")
            report_content.append("TOP REASONS FOR SUSPICIOUS CLASSIFICATION:")
            for reason, count in reason_counter.most_common(10):
                report_content.append(f"  • {reason}: {count:,} occurrences")
    
    # Risk Assessment
    report_content.append("")
    report_content.append("RISK ASSESSMENT")
    report_content.append("-" * 40)
    
    flags = detect_suspicious_patterns_with_details(timeline, user_phone)
    risk_score = min(len(flags) * 10, 100)
    
    report_content.append(f"Overall Risk Score: {risk_score}/100")
    
    if risk_score < 30:
        report_content.append("Risk Level: LOW")
    elif risk_score < 70:
        report_content.append("Risk Level: MEDIUM")
    else:
        report_content.append("Risk Level: HIGH")
    
    if flags:
        report_content.append("")
        report_content.append("DETECTED RED FLAGS / ANOMALIES:")
        for flag, explanation in flags:
            report_content.append(f"  ⚠ {flag}")
            report_content.append(f"    {explanation}")
    
    # Recommendations
    report_content.append("")
    report_content.append("INVESTIGATIVE RECOMMENDATIONS")
    report_content.append("-" * 40)
    
    if flags:
        report_content.append("1. Further investigation recommended for identified anomalies")
        report_content.append("2. Review communications with top suspicious contacts")
        report_content.append("3. Analyze late-night and rapid-fire communications")
        if any('financial' in flag[0].lower() for flag in flags):
            report_content.append("4. Scrutinize financial-related communications")
        if any('international' in flag[0].lower() for flag in flags):
            report_content.append("5. Review international communications")
        if suspicious_events:
            report_content.append("6. Examine suspicious communications for potential illegal activities")
    else:
        report_content.append("No significant anomalies detected. Standard monitoring recommended.")
    
    # Save report
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    report_filename = f'forensic_report_{timestamp}.txt'
    
    with open(report_filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_content))
    
    print(f" Comprehensive forensic report saved to '{report_filename}'")
    
    # Export additional files
    export_timeline_csv(timeline)
    export_contacts_csv(contact_counts, contact_details, user_phone)
    export_summary_json(timeline, sms_data, call_data, email_data, contact_counts, contact_details, user_phone)

def export_timeline_csv(timeline):
    """Export timeline to CSV"""
    if not timeline:
        return
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'forensic_timeline_{timestamp}.csv'
    
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        fieldnames = ['timestamp', 'id', 'source', 'contact', 'type', 'content', 'forensic_tag', 'reasons']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        
        for event in timeline:
            row = event.copy()
            row['timestamp'] = event['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
            row['reasons'] = '; '.join(event.get('reasons', []))
            # Remove details field if present
            if 'details' in row:
                del row['details']
            writer.writerow(row)
    
    print(f" Detailed timeline saved to '{filename}'")

def export_contacts_csv(contact_counts, contact_details, user_phone=None):
    """Export contacts to CSV"""
    if not contact_counts:
        return
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'forensic_contacts_{timestamp}.csv'
    
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Contact', 'Is_User', 'Total_Interactions', 'SMS_Count', 'Call_Count', 
                        'Email_Sent_Count', 'Email_Received_Count', 'Last_Contact_Date'])
        
        for contact, total_count in contact_counts.most_common():
            details = contact_details.get(contact, {})
            sms_count = details.get('sms_count', 0)
            call_count = details.get('call_count', 0)
            email_sent = details.get('sent_email_count', 0)
            email_received = details.get('received_email_count', 0)
            
            # Check if this is user's number
            is_user = 'Yes' if user_phone and contact == user_phone else 'No'
            
            # Get last contact date
            last_dates = []
            if 'last_contact' in details:
                last_dates.append(details['last_contact'])
            if 'last_call' in details:
                last_dates.append(details['last_call'])
            if 'last_email_sent' in details:
                last_dates.append(details['last_email_sent'])
            if 'last_email_received' in details:
                last_dates.append(details['last_email_received'])
            
            last_contact = max(last_dates) if last_dates else 'Unknown'
            if last_contact != 'Unknown':
                last_contact = last_contact.strftime('%Y-%m-%d %H:%M:%S')
            
            writer.writerow([contact, is_user, total_count, sms_count, call_count, 
                           email_sent, email_received, last_contact])
    
    print(f" Contact analysis saved to '{filename}'")

def export_summary_json(timeline, sms_data, call_data, email_data, contact_counts, contact_details, user_phone=None):
    """Export summary data to JSON"""
    if not timeline:
        return
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'forensic_summary_{timestamp}.json'
    
    # Get suspicious events
    suspicious_tags = ['SUSPICIOUS', 'FINANCIAL', 'URGENT', 'INTERNATIONAL', 'EXTENDED_COMM', 'SPAM']
    suspicious_events = [e for e in timeline if e['forensic_tag'] in suspicious_tags]
    
    summary = {
        'report_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'user_phone': user_phone,
        'data_summary': {
            'total_events': len(sms_data) + len(call_data) + len(email_data),
            'sms_count': len(sms_data),
            'call_count': len(call_data),
            'email_count': len(email_data)
        },
        'timeline_summary': {
            'start_date': timeline[0]['timestamp'].strftime('%Y-%m-%d %H:%M:%S') if timeline else None,
            'end_date': timeline[-1]['timestamp'].strftime('%Y-%m-%d %H:%M:%S') if timeline else None,
            'total_events': len(timeline)
        },
        'contact_summary': {
            'unique_contacts': len(contact_counts),
            'top_contacts': dict(contact_counts.most_common(10))
        },
        'suspicious_analysis': {
            'total_suspicious': len(suspicious_events),
            'suspicious_rate': len(suspicious_events) / len(timeline) if timeline else 0,
            'categories': dict(Counter(e['forensic_tag'] for e in suspicious_events))
        },
        'risk_assessment': {
            'flags': [flag for flag, _ in detect_suspicious_patterns_with_details(timeline, user_phone)],
            'risk_score': min(len(detect_suspicious_patterns_with_details(timeline, user_phone)) * 10, 100)
        }
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, default=str)
    
    print(f" Summary data saved to '{filename}'")

# ============================================================================
# SIMPLIFIED VISUALIZATION FUNCTIONS
# ============================================================================

def check_matplotlib():
    """Check if matplotlib is installed and import it"""
    try:
        import matplotlib
        import matplotlib.pyplot as plt
        import seaborn as sns
        return True, plt, sns
    except ImportError:
        print(" Matplotlib/Seaborn not installed. Install with: pip install matplotlib seaborn")
        return False, None, None

def create_jupyter_dashboard(timeline):
    """Create interactive dashboard in Jupyter"""
    if not IN_JUPYTER:
        return
    
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # Create interactive controls
    date_range = widgets.DateRangePicker(
        description='Date Range',
        value=(timeline[0]['timestamp'], timeline[-1]['timestamp'])
    )
    
    risk_filter = widgets.Dropdown(
        options=['All', 'High Risk', 'Medium Risk', 'Low Risk'],
        value='All',
        description='Risk Level:'
    )
    
    def update_dashboard(change):
        clear_output(wait=True)
        # Filter and display based on selections
        filtered = filter_timeline(timeline, date_range.value, risk_filter.value)
        display_timeline_events(filtered)
    
    # Display controls
    display(date_range, risk_filter)
    date_range.observe(update_dashboard, names='value')
    risk_filter.observe(update_dashboard, names='value')

def create_simple_visualizations(timeline, sms_data, call_data, email_data):
    """Create simple visualizations"""
    print("\n" + "-"*70)
    print(" CREATING VISUALIZATIONS")
    print("-"*70)
    
    if not timeline:
        print(" No data available for visualization.")
        return
    
    # Check for matplotlib
    matplotlib_available, plt, sns = check_matplotlib()
    if not matplotlib_available:
        return
    
    plt.style.use('seaborn-v0_8-darkgrid')
    
    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(timeline)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.day_name()
    
    # Get current timestamp for filenames
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    try:
        # 1. Communication Source Distribution
        print("\n1. Communication Source Distribution...")
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        source_counts = df['source'].value_counts()
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
        
        # Pie chart
        ax1.pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%',
                colors=colors[:len(source_counts)], startangle=90)
        ax1.set_title('Communication Source Distribution', fontsize=12, fontweight='bold')
        
        # Bar chart
        bars = ax2.bar(range(len(source_counts)), source_counts.values, 
                       color=colors[:len(source_counts)], alpha=0.8)
        ax2.set_title('Communication Source Counts', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Source Type')
        ax2.set_ylabel('Number of Events')
        ax2.set_xticks(range(len(source_counts)))
        ax2.set_xticklabels(source_counts.index, rotation=0)
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height):,}', ha='center', va='bottom', fontsize=10)
        
        plt.suptitle('Communication Sources Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'communication_sources_{timestamp}.png', dpi=300, bbox_inches='tight')
        print(f"   Saved: communication_sources_{timestamp}.png")
        if IN_JUPYTER:
            plt.show()
        else:
            plt.close()
        
        # 2. Hourly Activity Pattern
        print("\n2. Hourly Activity Pattern...")
        fig, ax = plt.subplots(figsize=(12, 6))
        
        hourly_counts = df['hour'].value_counts().sort_index()
        bars = ax.bar(hourly_counts.index, hourly_counts.values, alpha=0.7, color='#4ECDC4')
        
        ax.set_title('Hourly Communication Activity', fontsize=14, fontweight='bold')
        ax.set_xlabel('Hour of Day', fontsize=12)
        ax.set_ylabel('Number of Events', fontsize=12)
        ax.set_xticks(range(0, 24, 2))
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(f'hourly_activity_{timestamp}.png', dpi=300, bbox_inches='tight')
        print(f"   Saved: hourly_activity_{timestamp}.png")
        if IN_JUPYTER:
            plt.show()
        else:
            plt.close()
        
        # 3. Day of Week Analysis
        print("\n3. Day of Week Analysis...")
        fig, ax = plt.subplots(figsize=(10, 6))
        
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        day_counts = df['day_of_week'].value_counts().reindex(day_order)
        
        bars = ax.bar(range(len(day_counts)), day_counts.values, alpha=0.7, color='#FF6B6B')
        ax.set_title('Activity by Day of Week', fontsize=14, fontweight='bold')
        ax.set_xlabel('Day of Week', fontsize=12)
        ax.set_ylabel('Number of Events', fontsize=12)
        ax.set_xticks(range(len(day_counts)))
        ax.set_xticklabels([d[:3] for d in day_counts.index], rotation=0)
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(f'day_of_week_{timestamp}.png', dpi=300, bbox_inches='tight')
        print(f"   Saved: day_of_week_{timestamp}.png")
        if IN_JUPYTER:
            plt.show()
        else:
            plt.close()
        
        # 4. Forensic Category Distribution
        print("\n4. Forensic Category Distribution...")
        fig, ax = plt.subplots(figsize=(12, 6))
        
        category_counts = df['forensic_tag'].value_counts()
        
        # Sort categories by count
        categories_sorted = category_counts.sort_values(ascending=False)
        
        bars = ax.bar(range(len(categories_sorted)), categories_sorted.values, alpha=0.7, color='#45B7D1')
        ax.set_title('Forensic Category Distribution', fontsize=14, fontweight='bold')
        ax.set_xlabel('Forensic Category', fontsize=12)
        ax.set_ylabel('Number of Events', fontsize=12)
        ax.set_xticks(range(len(categories_sorted)))
        ax.set_xticklabels(categories_sorted.index, rotation=45, ha='right')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{int(height):,}', ha='center', va='bottom', fontsize=9)
        
        plt.tight_layout()
        plt.savefig(f'forensic_categories_{timestamp}.png', dpi=300, bbox_inches='tight')
        print(f"   Saved: forensic_categories_{timestamp}.png")
        if IN_JUPYTER:
            plt.show()
        else:
            plt.close()
        
        print(f"\n All visualizations saved with timestamp: {timestamp}")
        
    except Exception as e:
        print(f" Visualization error: {e}")
        import traceback
        traceback.print_exc()

# ============================================================================
# MAIN FUNCTION WITH SIMPLIFIED MENU
# ============================================================================

def main_jupyter():
    """Jupyter-optimized main function"""
    if not IN_JUPYTER:
        print("Not running in Jupyter. Use main() instead.")
        return
    
    from IPython.display import display, Markdown, HTML
    
    display(Markdown("# 🔍 Digital Forensics Analysis"))
    
    # Load data
    sms_data, call_data, email_data = data()
    
    if not any([sms_data, call_data, email_data]):
        display(Markdown("## No data loaded"))
        return
    
    # Find user
    user_phone = find_user_phone_number(sms_data, call_data)
    
    # Create timeline
    timeline = create_timeline(sms_data, call_data, email_data, user_phone)
    
    # Display summary
    display_summary_jupyter(timeline, user_phone)
    
    # Create dashboard
    create_jupyter_dashboard(timeline)

def main():
    """Main function with simplified menu"""
    print("\n" + "="*70)
    print(" MULTI SOURCE EVENT RECONSTRUCTION SYSTEM ")
    print("="*70)
    
    try:
        # Load data
        sms_data, call_data, email_data = data()
        
        if not sms_data and not call_data and not email_data:
            print("\n No data available. Exiting.")
            return
        
        # Find user's phone number from data
        print("\n" + "-"*70)
        print(" USER IDENTIFICATION")
        print("-"*70)
        user_phone = find_user_phone_number(sms_data, call_data)
        
        if user_phone:
            print(f"\n✅ USER'S PHONE NUMBER IDENTIFIED: {user_phone}")
            print(f"   Based on communication frequency and patterns")
        else:
            print("\n⚠️  User's phone number could not be identified from data.")
            print("   Analysis will continue with inferred relationships.")
            user_phone = None
        
        # Create timeline with user's phone number
        print("\n" + "-"*70)
        print(" CREATING UNIFIED TIMELINE")
        print("-"*70)
        timeline = create_timeline(sms_data, call_data, email_data, user_phone)
        
        if not timeline:
            print(" Failed to create timeline. Exiting.")
            return
        
        # Display total events first
        total_events = len(timeline)
        print(f"\n📊 TOTAL COMMUNICATION EVENTS LOADED: {total_events:,}")
        
        # Show contact summary
        contact_counts, _ = extract_contacts(sms_data, call_data, email_data)
        print(f"👥 UNIQUE CONTACTS IDENTIFIED: {len(contact_counts):,}")
        
        if user_phone:
            print(f"📱 USER'S PHONE NUMBER: {user_phone}")
        
        # Initial analysis
        print("\n" + "-"*70)
        print(" INITIAL ANALYSIS")
        print("-"*70)
        
        # Count suspicious communications
        suspicious_tags = ['SUSPICIOUS', 'FINANCIAL', 'URGENT', 'INTERNATIONAL', 'EXTENDED_COMM', 'SPAM']
        suspicious_events = [e for e in timeline if e['forensic_tag'] in suspicious_tags]
        print(f"🔍 SUSPICIOUS COMMUNICATIONS: {len(suspicious_events):,} ({len(suspicious_events)/total_events*100:.1f}%)")
        
        # Show top suspicious reasons
        if suspicious_events:
            reason_counter = Counter()
            for event in suspicious_events:
                for reason in event.get('reasons', []):
                    reason_counter[reason] += 1
            
            if reason_counter:
                print(f"   Top suspicious reasons:")
                for reason, count in reason_counter.most_common(3):
                    print(f"      • {reason}: {count:,}")
        
        # Main menu loop
        while True:
            print("\n" + "="*70)
            print(" MAIN MENU")
            print("="*70)
            print("1. Timeline Events (Show all communications)")
            print("2. View Detailed Analysis")
            print("3. Suspicious Communications Analysis")
            print("4. Multi Channel Pattern Detection")
            print("5. Visualize Data Patterns")
            print("6. Export the Report")
            print("7. Exit")
            print("="*70)
            
            choice = input("\nSelect option (1-7): ").strip()
            
            if choice == '1':
                display_timeline_events(timeline, user_phone)
            
            elif choice == '2':
                view_detailed_analysis(timeline, sms_data, call_data, email_data, user_phone)
            
            elif choice == '3':
                suspicious_communications_analysis(timeline, user_phone)

            elif choice == '4':
                coordinated_patterns = detect_coordinated_communications(timeline, user_phone)
                if coordinated_patterns:
                 print(f"\n🚨 **DETECTED {len(coordinated_patterns)} MULTI-CHANNEL PATTERNS**")
        
                # Initialize counters
                high_risk = []
                medium_risk = []
        
                # Group by risk level
                for pattern in coordinated_patterns:
                  if pattern['risk_level'] == 'HIGH':
                      high_risk.append(pattern)
                  else:
                      medium_risk.append(pattern)
        
                      print(f"\n📊 **RISK DISTRIBUTION:**")
                      print(f"   🔴 HIGH RISK: {len(high_risk):,} patterns (SMS → Call → Email)")
                      print(f"   🟡 MEDIUM RISK: {len(medium_risk):,} patterns (2-channel coordination)")
        
                    # Show most suspicious pattern
                      if high_risk:
                             print(f"\n⚠️ **MOST SUSPICIOUS PATTERN DETECTED:**")
                             pattern = high_risk[0]
                             print(f"   Contact: {pattern['contact']}")
                             print(f"   Time: {pattern['time_start'].strftime('%Y-%m-%d %H:%M')}")
                             print(f"   Pattern: {' → '.join([seq[0] for seq in pattern['sequence']])}")
            
                      # Ask if user wants to see details
                      show_details = input("\nShow detailed pattern? (y/n): ").lower()
                      if show_details == 'y':
                         print(f"\n📋 **DETAILED SEQUENCE:**")
                         for i, (medium, direction, content) in enumerate(pattern['sequence'], 1):
                             dir_text = "User → Contact" if direction == 'FROM_USER' else "Contact → User"
                             print(f"   {i}. {medium.upper()}: {dir_text}")
                             print(f"      Content: {content}...")
                      else:
                         print(f"\n📋 **MEDIUM RISK PATTERNS:**")
                         for i, pattern in enumerate(medium_risk[:3], 1):
                             print(f"\n   {i}. Contact: {pattern['contact'][:30]}...")
                             print(f"      Time: {pattern['time_start'].strftime('%H:%M')} - {pattern['time_end'].strftime('%H:%M')}")
                             print(f"      Channels: {' → '.join(pattern['media_used'])}")
            
            elif choice == '5':
                create_simple_visualizations(timeline, sms_data, call_data, email_data)
            
            elif choice == '6':
                export_forensic_report(timeline, sms_data, call_data, email_data, user_phone)
            
            elif choice == '7':
                print("\n" + "="*70)
                print(" ANALYSIS COMPLETED")
                print("="*70)
                print("Thank you for using the System.")
                break
            
            else:
                print(" Invalid option. Please select 1-6.")
    
    except KeyboardInterrupt:
        print("\n\n Analysis interrupted by user.")
    except Exception as e:
        print(f"\n Unexpected error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


 MULTI SOURCE EVENT RECONSTRUCTION SYSTEM 

----------------------------------------------------------------------
 LOADING DATA FILES
----------------------------------------------------------------------

Found 6 data files in current directory:
  • CDR-Call-Details.csv (23,215,814 bytes) - CSV
  • email_data-metadata.json (498 bytes) - JSON
  • CDR-Call-Details-metadata.json (508 bytes) - JSON
  • SMS-Data-metadata.json (467 bytes) - JSON
  • SMS-Data.csv (18,404,963 bytes) - CSV
  • email_data.csv (37,864,825 bytes) - CSV

----------------------------------------------------------------------
 Loading SMS data from SMS-Data.csv...
  Detected CSV format with comma delimiter
  Found columns: ['Phone Number', 'Contact Name', 'Direction', 'Message', 'Date/Time', 'Category', 'Relationship', 'Read', 'Message_ID']
  Processed 10,000 SMS records...
  Processed 20,000 SMS records...
  Processed 30,000 SMS records...
  Processed 40,000 SMS records...
  Processed 50,000 SMS records...
  Proc


Select option (1-7):  1



------------------------------------------------------------------------------------------------------------------------
 TIMELINE EVENTS - ALL COMMUNICATIONS
------------------------------------------------------------------------------------------------------------------------

📊 TOTAL COMMUNICATION EVENTS: 538,658
📱 USER'S PHONE NUMBER: +912127404903

------------------------------------------------------------------------------------------------------------------------
 1. SMS MESSAGES
------------------------------------------------------------------------------------------------------------------------

Total SMS Messages: 185,000

------------------------------------------------------------------------------------------------------------------------
Time                 From                      →   To                        Direction  Category        Message                       
-------------------------------------------------------------------------------------------------


Select option (1-7):  2



------------------------------------------------------------------------------------------------------------------------
 DETAILED COMMUNICATION ANALYSIS
------------------------------------------------------------------------------------------------------------------------

📊 TOTAL COMMUNICATION EVENTS: 538,658
📱 USER'S PHONE NUMBER: +912127404903

------------------------------------------------------------------------------------------------------------------------
 1. COMMUNICATION FLOW ANALYSIS

TOP COMMUNICATION FLOWS:
----------------------------------------------------------------------------------------------------
Rank  From → To                                          Count      %       
----------------------------------------------------------------------------------------------------
1     +912127404903 → +912127404903                      15,249         2.8%
2     +912129834687 → +912127404903                      15,232         2.8%
3     +913103021018 → +912127404903

...


 SUSPICIOUS COMMUNICATIONS ANALYSIS
📱 USER'S PHONE NUMBER: +912127404903

🔍 SUSPICIOUS COMMUNICATIONS DETECTED: 223,699 of 538,658 total events
   • Suspicious Rate: 41.5%

------------------------------------------------------------------------------------------------------------------------
 DETAILED SUSPICIOUS COMMUNICATIONS BREAKDOWN
------------------------------------------------------------------------------------------------------------------------

Category                  Count % of Suspicious Top Reason                               Risk      
-----------------------------------------------------------------------------------------------
SUSPICIOUS              150,161           67.1% Very short call (<5 seconds) (37794x)    HIGH      
FINANCIAL                47,876           21.4% Financial: Contains 'payment' (16198x)   HIGH      
SPAM                     21,585            9.6% Phishing: Contains 'update' (10590x)     LOW       
URGENT                    2,162          


Select option (1-7):  4



 MULTI-CHANNEL COMMUNICATION ANALYSIS

✅ No coordinated multi-channel patterns detected.

 MAIN MENU
1. Timeline Events (Show all communications)
2. View Detailed Analysis
3. Suspicious Communications Analysis
4. Multi Channel Pattern Detection
5. Visualize Data Patterns
6. Export the Report
7. Exit



Select option (1-7):  7



 ANALYSIS COMPLETED
Thank you for using the System.
